Importing libraries and modules

In [ ]:
import numpy as np

import pandas as pd
from pandas_datareader import data as web

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode()

import warnings
warnings.simplefilter("ignore")

from datetime import date, timedelta
import time

import yfinance as yf

import cufflinks as cf
cf.go_offline()

from scipy.signal import argrelextrema
from scipy.stats import linregress


Stock input and fetching its data

In [ ]:
stock_name = input("Enter the stock name\n")
stock_data = yf.download(tickers = stock_name + '.NS' ,period = '1mo', interval = '5m')
stock_data

Plotting Candlestick for the stock

In [ ]:
x_axis = stock_data.index

opens = stock_data['Open']
low = stock_data['Low']
high = stock_data['High']
close = stock_data['Adj Close']
volume = stock_data['Volume']

stock_candle = go.Candlestick(x=x_axis, open=opens, low=low,
                           high=high, close=close, name=stock_name+ '_candle')

stock_volume = go.Bar(x=x_axis, y=volume, name=stock_name + '_volume' )

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(trace=stock_candle, secondary_y=True)
fig.add_trace(trace=stock_volume, secondary_y=False)


fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=15,
                     label="15M",
                     step="minute",
                     stepmode="backward"),
                dict(count=45,
                     label="45M",
                     step="minute",
                     stepmode="backward"),
                dict(count=1,
                     label="1HR",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="1D",
                     step="day",
                     stepmode="todate"),
                dict(count=3,
                     label="3D",
                     step="day",
                     stepmode="todate"),
                dict(count=7,
                     label="1W",
                     step="day",
                     stepmode="backward"),
                dict(count=14,
                     label="2W",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1M",
                     step="month",
                     stepmode="backward"),
                dict(label="All", step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.update_layout(xaxis_title='Date',
                  yaxis_title="Price",
                  title=stock_name,
                  width=1250, height=800)

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=["2022-01-26", "2022-03-01", "2022-03-18", 
                    "2022-04-14", "2022-04-15", "2022-05-03", 
                    "2022-08-09", "2022-08-15", "2022-08-31", 
                    "2022-10-05", "2022-10-24", "2022-10-26", 
                    "2022-11-08"]),                    
        dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)
fig.show()

INDICATORS

In [ ]:
stock_data = yf.download(tickers = stock_name + '.NS' ,period = '10y', interval = '1d')
x_axis = stock_data.index

Simple Moving Average (SMA)

In [ ]:
# SMA Using cufflinks for golden cross and death cross

qf=cf.QuantFig(stock_data, title=stock_name,legend='top',name='cf_SMA')
qf.add_sma(periods=20, color='green')
qf.add_sma(periods=200, color='red')

qf.iplot()

In [ ]:
#SMA for price and volume

x = int(input("Shorter Period for SMA\n") or 50)
y = int(input("Longer Period for SMA\n") or 200)
z = int(input("Period for average Volume\n") or 10)

MAx_data = stock_data['Close'].rolling(x).mean()
MAy_data = stock_data['Close'].rolling(y).mean()
price_data = stock_data['Close']
Volume_MA_data = stock_data['Volume'].rolling(z).mean()

MAx = go.Scatter(x=x_axis, y=MAx_data, line=dict(color='green', width=1), name="SMA"+str(x))
MAy = go.Scatter(x=x_axis, y=MAy_data, line=dict(color='red', width=1), name="SMA"+str(y))
price = go.Scatter(x=x_axis, y=price_data, line=dict(color='orange', width=1), name="price")
stock_volume = go.Bar(x=x_axis, y=stock_data['Volume'], marker_color='blue',name=stock_name + '_volume' )
MA_volume = go.Scatter(x=x_axis, y=Volume_MA_data, line=dict(color='black', width=1), name="Volume_SMA"+str(z))


stock_data["SMA"+str(x)] = MAx_data
stock_data["SMA"+str(y)] = MAy_data
stock_data["Volume_SMA"+str(z)] = Volume_MA_data

fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)


fig.add_trace(trace=stock_volume, secondary_y = False)
fig.add_trace(trace=MAx, secondary_y = True)
fig.add_trace(trace=MAy, secondary_y = True)
fig.add_trace(trace=price, secondary_y = True)
fig.add_trace(trace = MA_volume, secondary_y=False)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

Exponential Moving Average (EMA)

In [ ]:
#EMA with cufflinks 

qf=cf.QuantFig(stock_data, title=stock_name,legend='top',name='cf_EMA')
qf.add_ema(periods=50, color='green')
qf.add_ema(periods=200, color='red')

qf.iplot()

In [ ]:
#EMA without cufflinks quant fig

x = int(input("Shorter Period for EMA\n") or 7)
y = int(input("Longer Period for EMA\n") or 21)

EMAx_data = stock_data['Close'].ewm(span=x, adjust=False).mean()
EMAy_data = stock_data['Close'].ewm(span=y, adjust=False).mean()
price_data = stock_data['Close']

stock_data["EMA"+str(x)] = EMAx_data
stock_data["EMA"+str(y)] = EMAy_data

EMAx = go.Scatter(x=x_axis, y=EMAx_data, line=dict(color='green', width=1), name="EMA"+str(x))
EMAy = go.Scatter(x=x_axis, y=EMAy_data, line=dict(color='red', width=1), name="EMA"+str(y))
price = go.Scatter(x=x_axis, y=price_data, line=dict(color='orange', width=1), name="price")

fig = go.Figure()

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                        
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.add_trace(trace=EMAx)
fig.add_trace(trace=EMAy)
fig.add_trace(trace=price)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

Relative Strength Index (RSI)

In [ ]:
#RSI with cufflinks

qf=cf.QuantFig(stock_data, title=stock_name,legend='top',name='cf_RSI')
qf.add_rsi(periods=14)

qf.iplot()


In [ ]:
#RSI without library

def RSI(df,period):
    temp_df = pd.DataFrame()
    temp_df = df
    temp_df['Price_Change'] = temp_df['Close'] - temp_df['Close'].shift(1)
    temp_df['Daily_gain'] = np.where(temp_df['Price_Change']>=0,temp_df['Price_Change'],0)
    temp_df['Daily_loss'] = np.where(temp_df['Price_Change']<0,abs(temp_df['Price_Change']),0)
    temp_df['Avg_gain'] = temp_df['Daily_gain'].rolling(period).sum()
    temp_df['Avg_loss'] = temp_df['Daily_loss'].rolling(period).sum()
    temp_df['RS'] = temp_df['Avg_gain']/temp_df['Avg_loss']
    temp_df['RSI'] = (temp_df['RS']/(1+temp_df['RS']))*100
    df['RSI_'+str(period)] = temp_df['RSI']
        
RSI(stock_data,14)

candle = go.Candlestick(x=x_axis,
                       open=stock_data['Open'],
                       low=stock_data['Low'],
                       high=stock_data['High'],
                       close=stock_data['Close'])
    
RSI_data = stock_data['RSI_14']

RSI_curve = go.Scatter(x=x_axis, y=RSI_data, 
                      line=dict(color='blue', width=1), name="RSI")

fig = go.Figure()
fig.add_trace(RSI_curve)
#fig.add_trace(trace=candle)
                                              
fig.update_xaxes(
    rangeslider_visible=True)
fig.update_yaxes(title="RSI")

fig.update_layout(height=500, width=1200)

a = int(input("Enter overbought region") or 80)
b = int(input("Enter oversold region") or 20)

fig.add_hline(y=b, line_width=1, line_dash="dash", line_color="red")
fig.add_hline(y=a, line_width=1, line_dash="dash", line_color="red")

fig.show()


MACD

In [ ]:
# MACD with cufflinks

qf=cf.QuantFig(stock_data, title=stock_name,legend='top',name='cf_RSI')
qf.add_macd(fast_period=12, slow_period=26, signal_period=9)

qf.iplot()


In [ ]:
# MACD without cufflinks

sp = int(input("Shorted Period\n") or 12)
lp = int(input("Longer Period\n") or 26)
signal = int(input("Signal Period\n") or 9)

ema_sp = stock_data['Close'].ewm(span = sp, adjust=False).mean()
ema_lp = stock_data['Close'].ewm(span = lp, adjust=False).mean()

macd_data = ema_sp - ema_lp
#macd_data.drop(macd_data.index[0:lp], inplace=True)

stock_data['MACD'+str(sp)+'_'+str(lp)] = macd_data

signal_line_data = macd_data.ewm(span = signal, adjust=False).mean()

stock_data['MACD_Signal_'+str(signal)] = signal_line_data

macd = go.Scatter(x = x_axis, y = macd_data, line=dict(color='green', width=1), name="macd")
signal_line = go.Scatter(x = x_axis, y = signal_line_data, line=dict(color='red', width=1), name="signal line")

fig = go.Figure()

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                          #for holidays
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.add_trace(trace=macd)
fig.add_trace(trace=signal_line)
fig.update_xaxes(rangeslider_visible=True)
fig.show()


In [ ]:
# MACD using ta library

from ta.trend import MACD

sp = int(input("Shorted Period\n") or 12)
lp = int(input("Longer Period\n") or 26)
signal = int(input("Signal Period\n") or 9)

#creates instance of the class MACD

macdi = MACD(close = stock_data['Adj Close'],
            window_slow=sp,
            window_fast=lp,
            window_sign=9)

fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.01)

macd_curve = go.Scatter(x=x_axis,y=macdi.macd(), line=dict(color = 'green', width=2), name = 'macd')
macd_signal = go.Scatter(x=x_axis,y=macdi.macd_signal(), line=dict(color = 'red', width=2), name = 'signal')
macd_diff = go.Bar(x=x_axis,y=macdi.macd_diff(), marker=dict(color='black'), name = 'macd_diff')

fig.add_trace(trace = macd_curve, row=1, col=1)
fig.add_trace(trace = macd_signal, row=1, col=1)
fig.add_trace(trace = macd_diff, row=1, col=1)

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                          #for holidays
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.update_layout(height=500, width=1200)

Bollinger Bands

In [ ]:
#Bollinger bands using Cufflinks

qf=cf.QuantFig(stock_data, title=stock_name,legend='top',name='cf_RSI')
qf.add_bollinger_bands(periods=20, boll_std=2)

qf.iplot()

In [ ]:
#Bollinger bands without library


x = int(input("Enter the period for Bollinger Bands\n") or 20)
y = float(input("How many times the standard deviation do you want to enclose the band within?") or 2)

SMA = stock_data['Close'].rolling(x).mean()
SD = stock_data['Close'].rolling(x).std(ddof=1)


plus_SD = SMA + (SD*y) 
minus_SD = SMA - (SD*y) 

candle = go.Candlestick(x=x_axis,
                       open=stock_data['Open'],
                       low=stock_data['Low'],
                       high=stock_data['High'],
                       close=stock_data['Close'])

Upper_band = go.Scatter(x=x_axis, y=plus_SD, line=dict(color='Blue', width=1), name="Upper Band")
Lower_band = go.Scatter(x=x_axis, y=minus_SD, line=dict(color='Blue', width=1), name="Lower Band")

fig =go.Figure()

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                      
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.add_trace(candle)
fig.add_trace(trace=Upper_band)
fig.add_trace(trace=Lower_band)
fig.show()


ICHIMOKU

In [ ]:
#for future days:

ichimoku_df = yf.download(tickers = stock_name + '.NS' ,period = '10y', interval = '1d')

temp_table = [np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]

date_series = pd.Series(stock_data.index)
sdate = date_series.iloc[date_series.shape[0]-1]
edate = sdate+timedelta(days=40)
delta = edate - sdate     
temp_list = []

for i in range(1,delta.days + 1):
    day = sdate + timedelta(days=i)
    temp_list.append(day)
    
temp_series= pd.Series(temp_list)
date_ser = date_series.append(temp_series, ignore_index=True)

for i in range(26):
    ichimoku_df.at[ichimoku_df.shape[0]] = temp_table

ichimoku_df = ichimoku_df.reset_index()


#Tenkan San

ichimoku_df['high_nine'] = ichimoku_df['High'].rolling(9).max()
ichimoku_df['low_nine'] = ichimoku_df['Low'].rolling(9).min()
ichimoku_df['Tenkan_San'] = (ichimoku_df['high_nine'] + ichimoku_df['low_nine'])/2

#Kijun San

ichimoku_df['high_twenty_six'] = ichimoku_df['High'].rolling(26).max()
ichimoku_df['low_twenty_six'] = ichimoku_df['Low'].rolling(26).min()
ichimoku_df['Kijun_San'] = (ichimoku_df['high_twenty_six'] + ichimoku_df['low_twenty_six'])/2

#Leading lines

ichimoku_df['Senkou_A'] = ((ichimoku_df['Tenkan_San'] + ichimoku_df['Kijun_San'])/2).shift(26)

ichimoku_df['high_fifty_two'] = ichimoku_df['High'].rolling(52).max()
ichimoku_df['low_fifty_two'] = ichimoku_df['Low'].rolling(52).min()

ichimoku_df['Senkou_B'] = ((ichimoku_df['high_fifty_two'] + ichimoku_df['low_fifty_two'])/2).shift(26)

#Lagging line

ichimoku_df['Chikou'] = ichimoku_df['Close'].shift(-26)


#Candlesticks

candle = go.Candlestick(x=date_ser,
                       open=ichimoku_df['Open'],
                       low=ichimoku_df['Low'],
                       high=ichimoku_df['High'],
                       close=ichimoku_df['Close'],
                       name='Candles')


fig = go.Figure()
                              
Conversion_Line = go.Scatter(x=date_ser, y=ichimoku_df['Tenkan_San'], line=dict(color='blue',width=1), name='Tenkan_San')
Base_Line = go.Scatter(x=date_ser, y=ichimoku_df['Kijun_San'], line=dict(color='Brown',width=1), name='Kijun_San')
Leading_A = go.Scatter(x=date_ser, y=ichimoku_df['Senkou_A'], line=dict(color='green',width=1), name='Senkou_A')
Leading_B = go.Scatter(x=date_ser, y=ichimoku_df['Senkou_B'], line=dict(color='red',width=1), name='Senkou_B')
Lagging = go.Scatter(x=date_ser, y=ichimoku_df['Chikou'], line=dict(color='Black',width=1), name='Chikou')


fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                      
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.add_trace(trace=candle)
fig.add_trace(trace=Conversion_Line)
fig.add_trace(trace=Base_Line)
fig.add_trace(trace=Leading_A)
fig.add_trace(trace=Leading_B)
fig.add_trace(trace=Lagging)

fig.show()

In [ ]:
#Stochastic Oscillator without library.

x = int(input('Enter the period for Stochastic Oscillator\n') or 14)
b = int(input('Enter overbought region\n') or 80)
a = int(input('Enter oversold region\n') or 20)

high_val = stock_data['High'].rolling(window=x).max()
low_val = stock_data['Low'].rolling(window=x).min()
curr_val = stock_data['Close']
so_num = curr_val - low_val
so_den = high_val- low_val
so = (so_num/so_den)*100

fig = go.Figure()

so_curve = go.Scatter(x=x_axis,y=so,
                     line=dict(color='red',width=1),
                     name='Stochastic_Curve')

fig.add_trace(trace=so_curve)
fig.add_hline(y=b, line_width=1, line_dash="dash", line_color="black")
fig.add_hline(y=a, line_width=1, line_dash="dash", line_color="black")

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                      
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.update_layout(height=500, width=900)

fig.show()


In [ ]:
#Stochastic using library

from ta.momentum import StochasticOscillator

stoch = StochasticOscillator(high = stock_data['High'],
                             low = stock_data['Low'],
                             close = stock_data['Close'],
                             window = 14,
                             smooth_window = 3)

fig = go.Figure()

stoch_curve = go.Scatter(x=x_axis,y=stoch.stoch(),
                        line=dict(color='red',width=1),
                        name='Stochastic Oscillator')

fig.add_trace(trace=stoch_curve)
fig.add_hline(y=b, line_width=1, line_dash="dash", line_color="black")
fig.add_hline(y=a, line_width=1, line_dash="dash", line_color="black")

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                      
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.update_layout(height=500, width=900)


fig.show()

In [ ]:
#ATR without library

x = int(input("Enter period for ATR") or 14)

curr_high = stock_data['High']
curr_low = stock_data['Low']
prev_close = stock_data['Close'].shift(1)

a = abs(curr_high-prev_close)
b = abs(curr_low-prev_close)
c = curr_high - curr_low
        
temp_df = pd.concat([a, b, c], axis=1)
temp_df['True_Range'] = temp_df.max(axis=1)
temp_df['ATR'] = temp_df['True_Range'].rolling(x).mean()

#adding data to our df
stock_data['True_Range'] = temp_df['True_Range']
stock_data['ATR'] = temp_df['ATR']

fig = go.Figure()

ATR_curve = go.Scatter(x=x_axis, y=temp_df['ATR'],
                      line=dict(color='red',width=1),
                      name='ATR')

fig.add_trace(trace=ATR_curve)

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                      
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.update_layout(height=400, width=1000)
fig.show()

In [ ]:
#ATR with library

qf = cf.QuantFig(stock_data, title=stock_name,legend='top',name='ATR')
qf.add_atr(periods=14)

qf.iplot()

In [ ]:
#ADX without library

temp_df = yf.download(tickers = stock_name + '.NS' ,period = '10y', interval = '1d')
temp_df['True_Range'] = stock_data['True_Range']
x_axis = temp_df.index

# temp_df = pd.DataFrame()
# temp_df = stock_data

# a = abs(stock_data['High'] - stock_data['Close'].shift(1))
# b = abs(stock_data['Low'] - stock_data['Close'].shift(1))
# c = stock_data['High'] - stock_data['Low']
# temp_df = pd.concat([a, b, c], axis=1)

#stock_data['True_Range'] = temp_df.max(axis=1)


temp_df['plus_DM'] = np.where(((temp_df['High'] - temp_df['High'].shift(1)) >
                                 (temp_df['Low'].shift(1) - temp_df['Low'])) &
                                ((temp_df['High'] - temp_df['High'].shift(1)) > 0),
                                (temp_df['High'] - temp_df['High'].shift(1)),0)

temp_df['minus_DM'] = np.where(((temp_df['High'] - temp_df['High'].shift(1)) <
                                 (temp_df['Low'].shift(1) - temp_df['Low'])) &
                                ((temp_df['Low'].shift(1) - temp_df['Low']) > 0),
                                (temp_df['Low'].shift(1) - temp_df['Low']),0)


def wilder_smoothening(name):
    smooth_it = 'smooth_'+name
    first_smooth = temp_df[name].head(14).sum()

    temp_df[smooth_it] = np.nan

    temp_df.at[temp_df.index[14],smooth_it] = first_smooth

    temp = first_smooth

    for i in range(15,temp_df.shape[0]):
        temp_df.at[temp_df.index[i],smooth_it] = temp - temp/14 + temp_df.loc[temp_df.index[i],name]
        temp = temp_df.loc[temp_df.index[i],smooth_it]

wilder_smoothening('True_Range')
wilder_smoothening('plus_DM')
wilder_smoothening('minus_DM')


temp_df['plus_DI'] = (temp_df['smooth_plus_DM']/temp_df['smooth_True_Range'])*100
temp_df['minus_DI'] = (temp_df['smooth_minus_DM']/temp_df['smooth_True_Range'])*100

temp_df['DX'] = ((abs(temp_df['plus_DI'] - temp_df['minus_DI']))/
                   (temp_df['plus_DI'] + temp_df['minus_DI'])) * 100

temp_sum = 0
for i in range(15,28):
    temp_sum += temp_df.loc[temp_df.index[i],'DX']

first_ADX = temp_sum/14

temp_df['ADX'] = np.nan

temp_df.at[temp_df.index[28],'ADX'] = first_ADX

temp = first_ADX

for i in range(29,temp_df.shape[0]):
    temp_df.at[temp_df.index[i],'ADX'] = ((temp*13) + temp_df.loc[temp_df.index[i],'DX'])/14
    temp = temp_df.loc[temp_df.index[i],'ADX']
    
#adding data to our df
stock_data['plus_DI'] = temp_df['plus_DI']
stock_data['minus_DI'] = temp_df['minus_DI']
stock_data['ADX'] = temp_df['ADX']
    
fig = go.Figure()
                              

plus_DI = go.Scatter(x=x_axis, y=temp_df['plus_DI'], line=dict(color='green',width=1), name='plus_DI')
minus_DI = go.Scatter(x=x_axis, y=temp_df['minus_DI'], line=dict(color='red',width=1), name='minus_DI')
ADX = go.Scatter(x=x_axis, y=temp_df['ADX'], line=dict(color='Black',width=1), name='ADX')


fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                      
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.add_trace(trace=plus_DI)
fig.add_trace(trace=minus_DI)
fig.add_trace(trace=ADX)

fig.show()


In [ ]:
#ADX with library

qf = cf.QuantFig(stock_data, title=stock_name,legend='top',name='ADX')
qf.add_adx(periods=14)

qf.iplot()

In [ ]:
#CCI(Commodity Channel Indicator) without library 


x = int(input("Enter the period\n") or 20)

Typical_Price = (stock_data['High'] + 
                 stock_data['Low'] + 
                 stock_data['Close'])/3

TP_sma = Typical_Price.rolling(x).mean()
TP_std = Typical_Price.rolling(x).std()

CCI = (((Typical_Price) - (TP_sma))/(0.015*TP_std))

fig = go.Figure()

CCI_curve = go.Scatter(x=x_axis, y=CCI,
                      line=dict(color='blue',width=1),
                      name='CCI')

fig.add_trace(trace=CCI_curve)

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                      
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.update_layout(height=400, width=700)
fig.show()

In [ ]:
#CCI using library

qf = cf.QuantFig(stock_data, title=stock_name,legend='top',name='CCI')
qf.add_cci(periods=20)

qf.iplot()

In [ ]:
#Generating dataset for local maxima and minima to draw a trendline and identify uptrend or downtrend

x = int(input("Enter the periods for a trend to be considered\n") or 7)

local_maxima = np.array(argrelextrema(stock_data['High'].values, np.greater, order=x))
local_minima = np.array(argrelextrema(stock_data['Low'].values, np.less, order=x))

peaks_x_data = []
peaks_y_data = []
peaks_z_data = []

for i in range(local_maxima.size):
    peaks_x_data.append(stock_data.index[local_maxima[0,i]])
    peaks_y_data.append(stock_data.loc[stock_data.index[local_maxima[0,i]],'High'])
    peaks_z_data.append(1)
    
valleys_x_data = []   
valleys_y_data = []
valleys_z_data = []

for i in range(local_minima.size):
    valleys_x_data.append(stock_data.index[local_minima[0,i]])
    valleys_y_data.append(stock_data.loc[stock_data.index[local_minima[0,i]],'Low'])
    valleys_z_data.append(-1)
    
df = pd.DataFrame()

df['x_data'] = np.hstack((peaks_x_data, valleys_x_data))
df['y_data'] = np.hstack((peaks_y_data, valleys_y_data))
df['Peak(1)/Valley(-1)'] = np.hstack((peaks_z_data, valleys_z_data))

df.sort_values('x_data',inplace=True)

#adding data to our df

df.reset_index()
stock_data['Peak(1)/Valley(-1)'] = 0

for i in range(df.shape[0]):
    stock_data.at[df.loc[i,'x_data'],'Peak(1)/Valley(-1)'] = df.loc[i,'Peak(1)/Valley(-1)']

#plotting
candle = go.Candlestick(x=x_axis,
                       open=stock_data['Open'],
                       low=stock_data['Low'],
                       high=stock_data['High'],
                       close=stock_data['Close'],
                       name='Candles')

trends = go.Scatter(x=df['x_data'], y=df['y_data'], line=dict(color='black',width=1), name='Trendline')

fig = go.Figure()

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                      
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.add_trace(trace=candle)
fig.add_trace(trace=trends)

fig.show()



In [ ]:
#Trendline based on Close price


from scipy.signal import argrelextrema

x = int(input("Enter the periods for a trend to be considered\n") or 7)

local_maxima = np.array(argrelextrema(stock_data['High'].values, np.greater, order=x))
local_minima = np.array(argrelextrema(stock_data['Low'].values, np.less, order=x))

trendline_data = np.hstack((local_maxima, local_minima))
sorted_trendline_data = np.sort(trendline_data)

x_data = []
y_data = []

for i in range(sorted_trendline_data.size):
    x_data.append(stock_data.index[sorted_trendline_data[0,i]])
    y_data.append(stock_data.loc[stock_data.index[sorted_trendline_data[0,i]],'Close'])
    
candle = go.Candlestick(x=x_axis,
                       open=stock_data['Open'],
                       low=stock_data['Low'],
                       high=stock_data['High'],
                       close=stock_data['Close'],
                       name='Candles')

trends = go.Scatter(x=x_data, y=y_data, line=dict(color='black',width=1), name='Trendline')

fig = go.Figure()

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                      
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.add_trace(trace=candle)
fig.add_trace(trace=trends)

fig.show()



In [ ]:
#plotting Parabolic SAR

temp_df = yf.download(tickers = stock_name + '.NS' ,period = '10y', interval = '1d')
x_axis = temp_df.index

first_PSAR = temp_df.loc[temp_df.index[0],'Low']
first_EP = temp_df.loc[temp_df.index[0],'High']

temp_df[['PSAR','EP','EP_PSAR','Acc_EP_PSAR','Acc','Trend']] = np.nan

if(temp_df.loc[temp_df.index[0],'Close'] <= df['y_data'][0]):
    first_trend = 1
else:
    first_trend = 0
    

temp_df.at[temp_df.index[0],'PSAR'] = first_PSAR
temp_df.at[temp_df.index[0],'EP'] = first_EP
temp_df.at[temp_df.index[0],'Trend'] = first_trend
temp_df.at[temp_df.index[0],'Acc'] = 0.02
temp_df.at[temp_df.index[0],'EP_PSAR'] = first_EP - first_PSAR
temp_df.at[temp_df.index[0],'Acc_EP_PSAR'] = (first_EP - first_PSAR)*0.02
    

def psar(i):
    #PSAR
    temp_df.at[temp_df.index[i],'PSAR'] = temp_df.loc[temp_df.index[i-1],'PSAR'] + temp_df.loc[temp_df.index[i-1],'Acc_EP_PSAR']
    
    #Trend
    
    if(temp_df.loc[temp_df.index[i],'PSAR'] < temp_df.loc[temp_df.index[i],'High']):
        temp_df.at[temp_df.index[i],'Trend'] = 1
    elif(temp_df.loc[temp_df.index[i],'PSAR'] > temp_df.loc[temp_df.index[i],'Low']):
        temp_df.at[temp_df.index[i],'Trend'] = -1
    
    #EP
    
    if(temp_df.loc[temp_df.index[i],'Trend'] == 1):
        if(temp_df.loc[temp_df.index[i],'High']>temp_df.loc[temp_df.index[i-1],'EP']):
            temp_df.at[temp_df.index[i],'EP'] = temp_df.loc[temp_df.index[i],'High']
        else:
            temp_df.at[temp_df.index[i],'EP'] = temp_df.loc[temp_df.index[i-1],'EP']
    elif(temp_df.loc[temp_df.index[i],'Trend'] == -1):
        if(temp_df.loc[temp_df.index[i],'Low']<temp_df.loc[temp_df.index[i-1],'EP']):
            temp_df.at[temp_df.index[i],'EP'] = temp_df.loc[temp_df.index[i],'Low']
        else:
            temp_df.at[temp_df.index[i],'EP'] = temp_df.loc[temp_df.index[i-1],'EP']
    
    #EP_PSAR
    
    temp_df.at[temp_df.index[i],'EP_PSAR'] = temp_df.at[temp_df.index[i],'EP'] - temp_df.at[temp_df.index[i],'PSAR']
    
    #Acc
  
    if(temp_df.loc[temp_df.index[i],'Trend'] == temp_df.loc[temp_df.index[i-1],'Trend']):
        if(temp_df.loc[temp_df.index[i],'Trend'] == 1):
            if(temp_df.loc[temp_df.index[i],'EP'] > temp_df.loc[temp_df.index[i-1],'EP']):
                if(temp_df.loc[temp_df.index[i-1],'Acc']>0.2):
                    temp_df.at[temp_df.index[i],'Acc'] = 0.2
                else:
                    temp_df.at[temp_df.index[i],'Acc'] = temp_df.loc[temp_df.index[i-1],'Acc'] + 0.02
            else:
                temp_df.at[temp_df.index[i],'Acc'] = temp_df.loc[temp_df.index[i-1],'Acc']

        if(temp_df.loc[temp_df.index[i],'Trend'] == -1):
            if(temp_df.loc[temp_df.index[i],'EP'] < temp_df.loc[temp_df.index[i-1],'EP']):
                if(temp_df.loc[temp_df.index[i-1],'Acc'] < 0.2):
                    temp_df.at[temp_df.index[i],'Acc'] = temp_df.loc[temp_df.index[i-1],'Acc'] + 0.02
                else:
                    temp_df.at[temp_df.index[i],'Acc'] = 0.2
            else:
                temp_df.at[temp_df.index[i],'Acc'] = temp_df.loc[temp_df.index[i-1],'Acc']
    else:
        temp_df.at[temp_df.index[i],'Acc'] = 0.02
    
    #Acc_EP_PSAR
    
    temp_df.at[temp_df.index[i],'Acc_EP_PSAR'] = temp_df.at[temp_df.index[i],'EP_PSAR'] * temp_df.at[temp_df.index[i],'Acc']
    

    
for x in range(1, temp_df.shape[0]):
    psar(x)

#adding data to our df
stock_data['PSAR'] = temp_df['PSAR']
stock_data['Trend'] = temp_df['Trend']
    
candle = go.Candlestick(x=x_axis,
                       open=temp_df['Open'],
                       low=temp_df['Low'],
                       high=temp_df['High'],
                       close=temp_df['Close'],
                       name='Candles')

PSAR_data = go.Scatter(x=x_axis, y=temp_df['PSAR'], line=dict(color='black',width=1), name='PSAR')

fig = go.Figure()

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                      
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

fig.add_trace(trace=candle)
fig.add_trace(trace=PSAR_data)

fig.show()
    

In [ ]:
#Horizontal line based on date

candle = go.Candlestick(x=x_axis,
                       open=stock_data['Open'],
                       low=stock_data['Low'],
                       high=stock_data['High'],
                       close=stock_data['Close'],
                       name='Candles')

fig = go.Figure()
fig.add_trace(trace=candle)
fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                      
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

answer='Y'

while(answer=='Y'):
    x = input("Enter date(YYYY-MM-DD)\n")

    date = x+' 00:00:00'

    fig.add_hline(y=stock_data.loc[date,'Close'], line_width=1, line_dash="solid", line_color="black")

    answer = input("Draw another line?(Y/N)\n")

fig.show()

In [ ]:
#Horizontal line based on price

candle = go.Candlestick(x=x_axis,
                       open=stock_data['Open'],
                       low=stock_data['Low'],
                       high=stock_data['High'],
                       close=stock_data['Close'],
                       name='Candles')

fig = go.Figure()
fig.add_trace(trace=candle)
fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), 
        dict(values=[])                      
        #dict(bounds=[15.5, 9.25], pattern="hour")
    ]
)

answer='Y'

while(answer=='Y'):
    x = input("Enter price.\n")

    fig.add_hline(y=x, line_width=1, line_dash="solid", line_color="black")

    answer = input("Draw another line?(Y/N)\n")

fig.show()

#CandleStick Patterns

In [ ]:
stock_data_indicator_df = stock_data
trend = stock_data['Trend']
stock_data = yf.download(tickers = stock_name + '.NS' ,period = '10y', interval = '1d')
stock_data['Trend'] = trend

In [ ]:
#Marubuzo

shadow_length = (float(input("Enter shadow range(%) for marubuzo\n") or 0.25))/100
max_candle_length = (float(input("Enter maximum candle length(%) for marubuzo\n") or 10))/100
min_candle_length = (float(input("Enter minimum candle length(%) for marubuzo\n") or 1))/100

stock_data['Bull_Maru'] = 0
stock_data['Bear_Maru'] = 0

def marubuzo(i,o,l,h,c):    
    if(c>o):
        #Bull Maru
        if(((h-c)/c < shadow_length) & ((o-l)/o < shadow_length) & (min_candle_length < (c-o)/o < max_candle_length)):
            stock_data.at[stock_data.index[i],'Bull_Maru'] = 1
    elif(o>c):
        #Bear Maru
        if(((c-l)/c < shadow_length) & ((h-o)/o < shadow_length) & (min_candle_length < (o-c)/c < max_candle_length)):
            stock_data.at[stock_data.index[i],'Bear_Maru'] = 1

for i in range(stock_data.shape[0]):
    marubuzo(i,
            stock_data.loc[stock_data.index[i],'Open'],
            stock_data.loc[stock_data.index[i],'Low'],
            stock_data.loc[stock_data.index[i],'High'],
            stock_data.loc[stock_data.index[i],'Close'])

stock_data.tail(50)
    

In [ ]:
#Spinning Top

shadow_length = (float(input("Enter minimum shadow range(%) for spinning top\n") or 2))/100
max_candle_length = (float(input("Enter maximum candle length(%) for spinning top\n") or 1))/100

stock_data['Spinning_Top'] = 0

def spinning_top(i,o,l,h,c):
    if(c>o):
        if(((c-o)/c <= max_candle_length) & ((h-c)/c >= shadow_length) & ((o-l)/o >= shadow_length)):
            stock_data.at[stock_data.index[i],'Spinning_Top'] = 1
    elif(o>c):
        if(((o-c)/o <= max_candle_length) & ((h-o)/o >= shadow_length) & ((c-l)/c >= shadow_length)):
            stock_data.at[stock_data.index[i],'Spinning_Top'] = 1

for i in range(stock_data.shape[0]):
    spinning_top(i,
            stock_data.loc[stock_data.index[i],'Open'],
            stock_data.loc[stock_data.index[i],'Low'],
            stock_data.loc[stock_data.index[i],'High'],
            stock_data.loc[stock_data.index[i],'Close'])
    
stock_data.tail(50)


In [ ]:
#paper umbrella (Hammer and Shooting Star)

max_candle_length = (float(input("Enter maximum candle length(%) for spinning top\n") or 2))/100
shooting_star_upper_shadow = float(input("Enter how many times the candle length ought the upper shadow of Shooting Star should be?\n") or 2)
shooting_star_lower_shadow = (float(input("Enter lower shadow range(%) for shooting star\n") or 0.5))/100
hammer_upper_shadow = (float(input("Enter upper shadow range(%) for hammer\n") or 0.5))/100
hammer_lower_shadow = float(input("Enter how many times the candle length ought the lower shadow of Hammer should be?\n") or 2)

stock_data['Shooting_Star'] = 0
stock_data['Hammer'] = 0

def paper_umbrella(i,o,l,h,c):
    if(stock_data.loc[stock_data.index[i],'Trend'] == 1):
        shooting_star(i,o,l,h,c)
    elif(stock_data.loc[stock_data.index[i],'Trend'] == 0):
        hammer(i,o,l,h,c)
        
def shooting_star(i,o,l,h,c):
    if((c>o) & (0.004 < (c-o)/o < max_candle_length) & ((h-c)>=(shooting_star_upper_shadow)*(c-o)) & ((o-l)/l < shooting_star_lower_shadow)):
        stock_data.at[stock_data.index[i],'Shooting_Star'] = 1
        
def hammer(i,o,l,h,c):
    if((0.004 < (o-c)/o < max_candle_length) & ((c-l)>=(hammer_lower_shadow)*(o-c)) & ((h-o)/o < hammer_upper_shadow)):
        stock_data.at[stock_data.index[i],'Hammer'] = 1
    

for i in range(stock_data.shape[0]):
    paper_umbrella(i,
            stock_data.loc[stock_data.index[i],'Open'],
            stock_data.loc[stock_data.index[i],'Low'],
            stock_data.loc[stock_data.index[i],'High'],
            stock_data.loc[stock_data.index[i],'Close'])
    
stock_data.tail(50)

In [ ]:
#Engulfing Pattern (Reversal)

stock_data['Bull_Engulf'] = 0
stock_data['Bear_Engulf'] = 0

def engulfing_pattern(i,o,c,o_1,c_1):
    if((stock_data.loc[stock_data.index[i],'Trend'] == 1) &
      (stock_data.loc[stock_data.index[i-1],'Trend'] == 1)):
        bear_engulf(i,o,c,o_1,c_1)
    elif((stock_data.loc[stock_data.index[i],'Trend'] == 0) & 
        (stock_data.loc[stock_data.index[i-1],'Trend'] == 0)):
        bull_engulf(i,o,c,o_1,c_1)
        
def bear_engulf(i,o,c,o_1,c_1):
    if((o>c) & (c_1>o_1)):
        if((o>c_1) & (c<o_1)):
            stock_data.at[stock_data.index[i],'Bear_Engulf'] = 1
            
def bull_engulf(i,o,c,o_1,c_1):
    if((c>o) & (c_1<o_1)):
        if((c>o_1) & (o<c_1)):
            stock_data.at[stock_data.index[i],'Bull_Engulf'] = 1
    
for i in range(1,stock_data.shape[0]):
    engulfing_pattern(i,
            stock_data.loc[stock_data.index[i],'Open'],
            stock_data.loc[stock_data.index[i],'Close'],
            stock_data.loc[stock_data.index[i-1],'Open'],
            stock_data.loc[stock_data.index[i-1],'Close'])

stock_data.tail(50)

In [ ]:
#Harami Reversal

harami_length = (float(input("Enter how much should the candle be contained(%)\n") or 30))/100

stock_data['Bull_Harami'] = 0
stock_data['Bear_Harami'] = 0

def harami(i,o,c,o_1,c_1):
    if((stock_data.loc[stock_data.index[i],'Trend'] == 1) &
      (stock_data.loc[stock_data.index[i-1],'Trend'] == 1)):
        bear_harami(i,o,c,o_1,c_1)
    elif((stock_data.loc[stock_data.index[i],'Trend'] == 0) & 
        (stock_data.loc[stock_data.index[i-1],'Trend'] == 0)):
        bull_harami(i,o,c,o_1,c_1)
        
def bear_harami(i,o,c,o_1,c_1):
    if((o>c) & (c_1>o_1)):
        if((o<c_1) & (c>o_1)):
            if((o-c)>(harami_length*(c_1-o_1))):
                stock_data.at[stock_data.index[i],'Bear_Harami'] = 1
            
def bull_harami(i,o,c,o_1,c_1):
    if((c>o) & (c_1<o_1)):
        if((o>c_1) & (c<o_1)):
            if((c-o)>(harami_length*(o_1-c_1))):
                stock_data.at[stock_data.index[i],'Bull_Harami'] = 1
    
for i in range(1,stock_data.shape[0]):
    harami(i,
            stock_data.loc[stock_data.index[i],'Open'],
            stock_data.loc[stock_data.index[i],'Close'],
            stock_data.loc[stock_data.index[i-1],'Open'],
            stock_data.loc[stock_data.index[i-1],'Close'])

stock_data.tail(50)

In [ ]:
#Gap up and Gap down  

stock_data['Gap_Up'] = 0
stock_data['Gap_Down'] = 0

gap_range = (float(input("Enter the gap range(%) wrt previous close price\n") or 0.5))/100

def gap(i,o,c,o_1,c_1):
    if((o>(c_1*gap_range+(c_1)))):
        if((c>o)):
            stock_data.at[stock_data.index[i],'Gap_Up'] = 1
        
    elif((o<(c_1-(c_1*gap_range)))):
        if((o>c)):
            stock_data.at[stock_data.index[i],'Gap_Down'] = 1
    

for i in range(stock_data.shape[0]):
    gap(i,
            stock_data.loc[stock_data.index[i],'Open'],
            stock_data.loc[stock_data.index[i],'Close'],
            stock_data.loc[stock_data.index[i-1],'Open'],
            stock_data.loc[stock_data.index[i-1],'Close'])

stock_data.tail(50)

In [ ]:
#Star Reversal pattern for smaller timeframe(1min,2min,5min...)

stock_data['Evening_Star'] = 0
stock_data['Morning_Star'] = 0

def star(i,o,c,o_1,c_1,o_2,c_2):
    if((stock_data.loc[stock_data.index[i],'Trend'] == 1) &
      (stock_data.loc[stock_data.index[i-1],'Trend'] == 1) &
      (stock_data.loc[stock_data.index[i-1],'Trend'] == 1)):
        evening_star(i,o,c,o_1,c_1,o_2,c_2)
        
def star(i,o,c,o_1,c_1,o_2,c_2):
    if((stock_data.loc[stock_data.index[i],'Trend'] == 0) &
      (stock_data.loc[stock_data.index[i-1],'Trend'] == 0) &
      (stock_data.loc[stock_data.index[i-1],'Trend'] == 0)):
        morning_star(i,o,c,o_1,c_1,o_2,c_2)

def evening_star(i,o,c,o_1,c_1,o_2,c_2):
    if((c_2>o_2) &
       (stock_data.loc[stock_data.index[i-1],'Gap_Up'] == 1) &
       (stock_data.loc[stock_data.index[i],'Gap_Down'] == 1) &
       (c<o_2)):
        stock_data.at[stock_data.index[i],'Evening_Star'] = 1
        
def morning_star(i,o,c,o_1,c_1,o_2,c_2):
    if((c_2<o_2) &
       (stock_data.loc[stock_data.index[i-1],'Gap_Down'] == 1) &
       (stock_data.loc[stock_data.index[i],'Gap_Up'] == 1) &
       (c>o_2)):
        stock_data.at[stock_data.index[i],'Morning_Star'] = 1
        
for i in range(stock_data.shape[0]):
    star(i,
            stock_data.loc[stock_data.index[i],'Open'],
            stock_data.loc[stock_data.index[i],'Close'],
            stock_data.loc[stock_data.index[i-1],'Open'],
            stock_data.loc[stock_data.index[i-1],'Close'],
            stock_data.loc[stock_data.index[i-2],'Open'],
            stock_data.loc[stock_data.index[i-2],'Close'])
    
stock_data.tail(50)

In [ ]:
stock_data_pattern_df = stock_data

Chart Patterns- Double Top, Double Bottom, Triple Top, Triple Bottom, Flag Up, Flag down, Triangle, Penant,etc.

In [ ]:
#RSI value around 41-59 indicate that during the period the market trended sideways with avg gain ~ avg loss
#RSI >= 60 implies an uptrend during the lookback period with avg gain > avg loss
#RSI <= 40 implies a downtrend during the lookback period with avg loss < avg gain

#Now a static RSI value over two period forms a Chart pattern with two symmetrical triangles.
#If the static RSI value over two period is around 41-59, look for double top or double bottom.
#If the static RSI value over two period is >=60, look for flag up pattern.
#If the static RSI value over two period is <=40, look for flag down pattern.

#we get RSI value for each period. if period=14, we get RSI for first 14 day and then for next 14 day one after other.
#Now if any two or more consecutive values same, we can proceed with the principles above.

date = []
period_RSI = []
x = int(input("Enter Period\n") or 14)
y = float(input("Enter RSI allowable fluctuation in detecting pattern\n") or 2 )
a = float(input("Enter RSI value above which you want to consider uptrend\n") or 57)
b = float(input("Enter RSI value below which you want to consider downtrend\n") or 43)

for i in range(0,x-1):
    for j in range(i,stock_data_indicator_df.shape[0],x):
        date.append(stock_data_indicator_df.index[j])
        period_RSI.append(stock_data_indicator_df.loc[stock_data_indicator_df.index[j],'RSI_'+str(x)])

RSI_df = pd.DataFrame(period_RSI,index=date,columns=['RSI_'+str(x)])

stock_data_pattern_df['Double_Top/Double_Bottom'] = 0
stock_data_pattern_df['Triple_Top/Triple_Bottom'] = 0
stock_data_pattern_df['Flag_Up/Flag_Down'] = 0

for i in range(RSI_df.shape[0]-2):
    curr_val = RSI_df.loc[RSI_df.index[i],'RSI_'+str(x)]
    next_val = RSI_df.loc[RSI_df.index[i+1],'RSI_'+str(x)]
    max_val = curr_val + y
    min_val = curr_val - y
    if((min_val <= next_val) & (next_val <= max_val)):
        if((next_val>=curr_val) &
           (next_val - (2*y) <= RSI_df.loc[RSI_df.index[i+2],'RSI_'+str(x)]) &
           (RSI_df.loc[RSI_df.index[i+2],'RSI_'+str(x)] <= next_val+y)):
            stock_data_pattern_df.at[RSI_df.index[i],'Triple_Top/Triple_Bottom'] = 1
        elif((next_val<curr_val) &
             (next_val-y <= RSI_df.loc[RSI_df.index[i+2],'RSI_'+str(x)]) &
             (RSI_df.loc[RSI_df.index[i+2],'RSI_'+str(x)]<= next_val+(2*y))):
            stock_data_pattern_df.at[RSI_df.index[i],'Triple_Top/Triple_Bottom'] = 1
        elif( (b <= curr_val) &
             (curr_val <= a)):
            stock_data_pattern_df.at[RSI_df.index[i],'Double_Top/Double_Bottom'] = 1
        elif(curr_val > a):
            stock_data_pattern_df.at[RSI_df.index[i],'Flag_Up/Flag_Down'] = 1
        elif(curr_val < b):
            stock_data_pattern_df.at[RSI_df.index[i],'Flag_Up/Flag_Down'] = -1
            
stock_data_pattern_df    


In [ ]:
filt = stock_data_pattern_df['Double_Top/Double_Bottom'] == 1
stock_data_pattern_df[filt]

Dataframe of all the days where we encountered a pattern

In [ ]:
filt = ((stock_data_pattern_df['Bull_Maru'] == 1) |
        (stock_data_pattern_df['Bear_Maru'] == 1) |
        (stock_data_pattern_df['Spinning_Top'] == 1) |
        (stock_data_pattern_df['Shooting_Star'] == 1) |
        (stock_data_pattern_df['Hammer'] == 1) |
        (stock_data_pattern_df['Bull_Engulf'] == 1) |
        (stock_data_pattern_df['Bear_Engulf'] == 1) |
        (stock_data_pattern_df['Bull_Harami'] == 1) |
        (stock_data_pattern_df['Bear_Harami'] == 1) |
        (stock_data_pattern_df['Gap_Up'] == 1) |
        (stock_data_pattern_df['Gap_Down'] == 1) |
        (stock_data_pattern_df['Evening_Star'] == 1) |
        (stock_data_pattern_df['Morning_Star'] == 1) |
        (stock_data_pattern_df['Double_Top/Double_Bottom'] == 1) |
        (stock_data_pattern_df['Triple_Top/Triple_Bottom'] == 1) |
        (stock_data_pattern_df['Flag_Up/Flag_Down'] == 1)
       )
pattern_signal_df = stock_data_pattern_df[filt]
pattern_signal_df

Machine Learning

In [ ]:
from scipy.stats import linregress

from sklearn import neighbors, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib

from xgboost import XGBClassifier,plot_importance

In [ ]:
#stock_data_indicator_df.dropna(inplace=True)
stock_data = yf.download(tickers = stock_name + '.NS' ,period = '10y', interval = '1d')

In [ ]:
def normalize(temp_series,name):
    stock_data['Normalized_'+name] = np.nan
    min_value = stock_data_indicator_df[name].min()
    max_value = stock_data_indicator_df[name].max()
    
    for i in range(stock_data_indicator_df.shape[0]):
        stock_data.at[stock_data_indicator_df.index[i],'Normalized_'+name] = ((stock_data_indicator_df.loc[stock_data_indicator_df.index[i],name]-min_value)/(max_value-min_value))*100

def covariance(series_x,series_y):
    temp_df = pd.concat([series_x,series_y],axis=1)
    temp_df.dropna(inplace=True)
    x_mean = series_x.mean()
    y_mean = series_y.mean()
    x_diff = series_x-x_mean
    y_diff = series_y-y_mean
    numerator = (x_diff*y_diff).sum()
    covariance = numerator/temp_df.shape[0]
    return covariance
    
def correlation(series_x,series_y):
    temp_df = pd.concat([series_x,series_y],axis=1)
    temp_df.dropna(inplace=True)
    x_mean = series_x.mean()
    y_mean = series_y.mean()
    x_diff = series_x-x_mean
    y_diff = series_y-y_mean
    numerator = (x_diff*y_diff).sum()
    covariance = numerator/temp_df.shape[0]
    corr_coeff = covariance/(series_x.std()*series_y.std())
    return corr_coeff
    

Checking covariance and correlation between indicator and close price

In [ ]:
normalize(stock_data_indicator_df['Close'],'Close')
normalize(stock_data_indicator_df['Open'],'Open')
indicator = ['RSI','ADX']
for i in indicator:
    normalize(stock_data_indicator_df[i],i)
    print("The covariance between "+i+" and today's Close Price is:\t")
    print(covariance(stock_data['Normalized_RSI'],stock_data['Normalized_Close']))
    print("The correlation between "+i+" and today's Close Price is:\t")
    print(correlation(stock_data['Normalized_RSI'],stock_data['Normalized_Close']))

As expected, there is no direct relation between the RSI or ADX and the Close price of the same day.

In [ ]:
indicator = ['RSI','ADX']
for i in indicator:
    normalize(stock_data_indicator_df[i],i)
    print("The covariance between "+i+" and future Close Price is:\t")
    print(covariance(stock_data['Normalized_RSI'],stock_data['Normalized_Open'].shift(-1)))
    print("The correlation between "+i+" and future Close Price is:\t")
    print(correlation(stock_data['Normalized_RSI'],stock_data['Normalized_Open'].shift(-1)))

There seems to be no direct relation between today's RSI or ADX wrt to the next day stock price.

Checking relation between indicator trend and price trend

In [ ]:
lists = ['Close','SMA50','RSI','ADX']

a = int(input('Period to compare indicator trend and price trend\n') or 5)

def get_slope(data):
    y = np.array(data)
    x = np.arange(len(y))
    m,c,r,p,error = linregress(x,y)
    return m
for i in lists:
    stock_data_indicator_df[i+'_Slope'] = stock_data_indicator_df[i].rolling(a).apply(get_slope,raw=True)
  
    

In [ ]:
for i in lists[1:]:
    print("The covariance between "+i+" slope and Close Price slope is:\t")
    print(covariance(stock_data_indicator_df[i+'_Slope'],stock_data_indicator_df['Close_Slope']))
    print("The correlation between "+i+" slope and Close Price slope is:\t")
    print(correlation(stock_data_indicator_df[i+'_Slope'],stock_data_indicator_df['Close_Slope']))

No relation found between indicator trend and price trend

Checking trend frequency before applying a model to it

In [ ]:
x = (int(input("Enter price range(%) for a trend to be considered\n") or 5))/100
y = float(input("Enter target/stoploss ratio\n") or 2)
z = int(input("Enter the period for a trend\n") or 15)

High_array = np.array(stock_data['High'])
Low_array = np.array(stock_data['Low'])

stock_data['Trade_Trend'] = np.nan

# 1  : Bullish target without hitting stoploss (Uptrend)
# -1 : Bearish target without hitting stoploss (Downtrend)
# 0  : Price between Bullish and Bearish stoploss bound (Sidways- don't trade here)
# 2  : Price hitting both Bullish and Bearish target (Volatile)
# -2 : Price exceeding both Bullish and Bearish stoploss without reaching target (worst case)
#  5 : Trend period not sufficient to conclude a trend

def trend(df):
    for i in range(stock_data.shape[0]):
        Bull_target = stock_data.loc[stock_data.index[i],'Close']*(1+x)
        Bear_target = stock_data.loc[stock_data.index[i],'Close']*(1-x)
        Bull_stoploss = (stock_data.loc[stock_data.index[i],'Close']*(1-(x/y)))
        Bear_stoploss = (stock_data.loc[stock_data.index[i],'Close']*(1+(x/y)))
        Period_max = High_array[i:i+z].max()
        Period_min = Low_array[i:i+z].min()
        
        if((Period_min <= Bear_target)&
            (Period_max<Bear_stoploss)):
            stock_data.at[stock_data.index[i],'Trade_Trend'] = -1
        elif((Period_max >= Bull_target)&
          (Period_min > Bull_stoploss)):
            stock_data.at[stock_data.index[i],'Trade_Trend'] = 1
        elif((Period_max<Bear_stoploss)&
            (Period_min>Bull_stoploss)):
            stock_data.at[stock_data.index[i],'Trade_Trend'] = 0
        elif((Period_max >= Bear_stoploss)&
            (Period_max < Bull_target)&
            (Period_min <= Bull_stoploss)&
            (Period_min > Bear_target)):
            stock_data.at[stock_data.index[i],'Trade_Trend'] = -2
        elif((Period_max >= Bull_target)&
            (Period_min <= Bear_target)):
            stock_data.at[stock_data.index[i],'Trade_Trend'] = 2
        else:
            stock_data.at[stock_data.index[i],'Trade_Trend'] = 5

trend(stock_data)            
        
plt.hist(stock_data['Trade_Trend'])
plt.grid()
plt.xlabel("Trade_Trend")
plt.ylabel("Trend Frequency")
plt.show()

Relation between indicator and trend

In [ ]:
up_filt = (stock_data['Trade_Trend'] == 1)
down_filt = (stock_data['Trade_Trend'] == -1)
side_filt = ((stock_data['Trade_Trend'] == 0) | (stock_data['Trade_Trend'] == -2))
up_RSI = list(stock_data[up_filt]['Normalized_RSI'])
down_RSI = list(stock_data[down_filt]['Normalized_RSI'])
side_RSI = list(stock_data[side_filt]['Normalized_RSI'])
plt.hist(up_RSI, bins=100, alpha=0.5, label='uptrend_RSI')
plt.hist(down_RSI, bins=100, alpha=0.5, label='downtrend_RSI')
plt.hist(side_RSI, bins=100, alpha=0.5, label='sidetrend_RSI')

plt.legend(loc='upper right')
plt.show()


As we can see above, there seems to be no distinction between RSI value during uptrend, downtrend or sidetrend.

In [ ]:
up_filt = (stock_data['Trade_Trend'] == 1)
down_filt = (stock_data['Trade_Trend'] == -1)
side_filt = ((stock_data['Trade_Trend'] == 0) | (stock_data['Trade_Trend'] == -2))
up_ADX = list(stock_data[up_filt]['Normalized_ADX'])
down_ADX = list(stock_data[down_filt]['Normalized_ADX'])
side_ADX = list(stock_data[side_filt]['Normalized_ADX'])
plt.hist(up_ADX, bins=100, alpha=0.5, label='uptrend_ADX')
plt.hist(down_ADX, bins=100, alpha=0.5, label='downtrend_ADX')
plt.hist(side_ADX, bins=100, alpha=0.5, label='sidetrend_ADX')

plt.legend(loc='upper right')
plt.show()

As we can see above, there seems to be no distinction between ADX value during uptrend, downtrend or sidetrend.

Relation between multiple indicator and trend

We have above defined categories for price trend(-2 to 2) and so we will use categorical model(KNN,XGBoost) to see relation between indicator and trend.

In [ ]:
backup_stock_data_indicator_df = stock_data_indicator_df.copy()

In [ ]:
stock_data_indicator_df['Trade_Trend'] = np.nan
for i in range(stock_data_indicator_df.shape[0]):
    stock_data_indicator_df.at[stock_data_indicator_df.index[i],'Trade_Trend'] = stock_data.loc[stock_data_indicator_df.index[i],'Trade_Trend']
    
stock_data_indicator_df['Trade_Trend'] = stock_data_indicator_df['Trade_Trend'].replace(np.nan,10)
stock_data_indicator_df.dropna(inplace=True)
#X = stock_data_indicator_df[['RSI','ADX']]
#y = stock_data_indicator_df['Trade_Trend']

X = stock_data_indicator_df[['RSI_Slope','ADX_Slope']]
y = stock_data_indicator_df['Trade_Trend']

#Random Sampling
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

#Sequential Sampling
train_index = int(0.8*len(X))
X_train,X_test = X[:train_index], X[train_index:]
y_train,y_test = y[:train_index], y[train_index:]

#KNN model
knn = neighbors.KNeighborsClassifier(n_neighbors=200,weights='uniform')
knn.fit(X_train,y_train)

train_prediction = knn.predict(X_train)
test_prediction = knn.predict(X_test)

accuracy_train = metrics.accuracy_score(y_train,train_prediction)
accuracy_test = metrics.accuracy_score(y_test,test_prediction)

#Checking overfitting

print("Train Accuracy:")
print(accuracy_train*100)
print("Test Accuracy:")
print(accuracy_test*100)

# 1: Comparing accuracy with label frequency

print("Label frequency to compare with accuracy")
print(stock_data_indicator_df['Trade_Trend'].value_counts()*100/stock_data_indicator_df['Trade_Trend'].count())

# 2: Comparing accuracy using Gambler test

gambler_prediction = np.random.choice([-2,-1,0,1,2], len(X_test))
gambler_accuracy = metrics.accuracy_score(gambler_prediction,y_test)
print("Gambler Accuracy:")
print(gambler_accuracy*100)


In [ ]:
knn.predict(X.tail(1))

In [ ]:
#XGB model

le = LabelEncoder()
y = le.fit_transform(y)
X = stock_data_indicator_df[['RSI_Slope','ADX_Slope','RSI','ADX','MACD12_26','ATR']]

#original ~ new
# -2 ~ 0
# -1 ~ 1
# 0 ~ 2
# 1 ~ 3
# 2 ~ 4
# 5 ~ 5

#Random Sampling
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

# #Sequential Sampling
# train_index = int(0.8*len(X))
# X_train,X_test = X[:train_index], X[train_index:]
# y_train,y_test = y[:train_index], y[train_index:]

xgb = XGBClassifier()
xgb.fit(X_train,y_train)

train_prediction = xgb.predict(X_train)
test_prediction = xgb.predict(X_test)

accuracy_train = metrics.accuracy_score(y_train,train_prediction)
accuracy_test = metrics.accuracy_score(y_test,test_prediction)

print("Train Accuracy:")
print(accuracy_train*100)
print("Test Accuracy:")
print(accuracy_test*100)
plot_importance(xgb)
plt.show()

In [ ]:
xgb.predict(X.tail(1))

ML stratergy. if todays data signals an uptrend, trade.

Creating a stratergy

- Creating an opportunity universe.

- To detect a signal.
Signal:
     - encountering a candle or a chart pattern
        
- Cross Checking Signal:
     - confirming volumes and indicators
        
Using ML model to look for future trend.

- Deciding Trade size based on number of signals received from patterns and indicators.

- Setting up Target and Stoploss

- Executing order.

- Updating order for trailing stoploss
    

Creating Opportunity Universe

In [ ]:
oppor_univ = pd.read_csv("https://www1.nseindia.com/content/indices/ind_nifty200list.csv")
oppor_univ = oppor_univ[['Company Name','Symbol']]
oppor_univ_symbols = oppor_univ['Symbol'].tolist()
oppor_univ_symbols

Signal Detection

In [ ]:
#patterns that a user wants to implement to filter down signals
patterns = stock_data_pattern_df.columns[7:]
t_patterns = []
for pattern in patterns:
    if((stock_data_pattern_df.loc[stock_data.tail(1).index[0],pattern] == 1)):
        t_patterns.append(pattern)
        
print("Pattern detected today:\n")
print(t_patterns)

#if signal detected, pass another signal to make the follow up code run.

Indicator confirmation

In [ ]:
stock_data_indicator_df.columns

In [ ]:
indicators_confirming = 0
indicators_confirmed = []

#Volume
total_indicators = 5
if((stock_data_indicator_df.loc[stock_data.tail(1).index[0],'Volume']) > (stock_data_indicator_df.loc[stock_data.tail(1).index[0],'Volume_SMA10'])):
    indicators_confirming += 1
    indicators_confirmed.append('Volume')
if((stock_data_indicator_df.loc[stock_data.tail(1).index[0],'EMA7'] > stock_data_indicator_df.loc[stock_data.tail(1).index[0],'EMA21'])):
    indicators_confirming += 1
    indicators_confirmed.append('EMA bull crossover')
if((stock_data_indicator_df.loc[stock_data.tail(1).index[0],'SMA50'] > stock_data_indicator_df.loc[stock_data.tail(1).index[0],'SMA200'])):
    indicators_confirming += 1
    indicators_confirmed.append('Golden Cross')
if((stock_data_indicator_df.loc[stock_data.tail(1).index[0],'MACD_Signal_9'] > stock_data_indicator_df.loc[stock_data.tail(1).index[0],'MACD12_26'])):
    indicators_confirming += 1
    indicators_confirmed.append('Bull MACD')
if((stock_data_indicator_df.loc[stock_data.tail(1).index[0],'Trend'] == 1)):
    indicators_confirming += 1
    indicators_confirmed.append('PSAR uptrend')
    
print(str(indicators_confirming)+'/'+str(total_indicators))
print(indicators_confirmed)

Backtesting Stratergy

STRATERGY 1: Backtesting whether a candlestick pattern alone serves as a good Trading Signal:

In [ ]:
def long_trade(j):
    i = j+1
    price = stock_data.loc[stock_data.index[i],'Open']
    target = price * (1+(target_p/100))
    stoploss = price * (1-(stoploss_p/100))
    Long_position_df.at[index,'StopLoss'] = stoploss
    Long_position_df.at[index,'Target'] = target
    max_period = i+max_holding_period
    while(price < target and price > stoploss and i<max_period):
        i += 1
        if(stock_data.loc[stock_data.index[i],'High'] > target):
            Long_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Long_position_df.at[index,'Exit_Price'] = target
            Long_position_df.at[index,'Holding_Period'] = (i - (max_period-max_holding_period))
            price = target
        elif(stock_data.loc[stock_data.index[i],'Low'] < stoploss):
            Long_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Long_position_df.at[index,'Exit_Price'] = stoploss
            Long_position_df.at[index,'Holding_Period'] = (i - (max_period-max_holding_period))
            price = stoploss
        else:
            Long_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Long_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i],'Close']
            Long_position_df.at[index,'Holding_Period'] = (i - (max_period-max_holding_period))
            price = stock_data.loc[stock_data.index[i],'Close']
    Long_position_df.at[index,'P/L'] = Long_position_df.loc[index,'Exit_Price'] - Long_position_df.loc[index,'Entry_Price']

def Short_trade(j):
    i = j+1
    price = stock_data.loc[stock_data.index[i],'Open']
    target = price * (1-(target_p/100))
    stoploss = price * (1+(stoploss_p/100))
    Short_position_df.at[index,'StopLoss'] = stoploss
    Short_position_df.at[index,'Target'] = target
    max_period = i+max_holding_period
    while(price > target and price < stoploss and i<max_period):
        i += 1
        if(stock_data.loc[stock_data.index[i],'High'] > stoploss):
            Short_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Short_position_df.at[index,'Exit_Price'] = stoploss
            Short_position_df.at[index,'Holding_Period'] = (i - (max_period-max_holding_period))
            price = stoploss
        elif(stock_data.loc[stock_data.index[i],'Low'] < target):
            Short_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Short_position_df.at[index,'Exit_Price'] = target
            Short_position_df.at[index,'Holding_Period'] = (i - (max_period-max_holding_period))
            price = target
        else:
            Short_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Short_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i],'Close']
            Short_position_df.at[index,'Holding_Period'] = (i - (max_period-max_holding_period))
            price = stock_data.loc[stock_data.index[i],'Close']
    Short_position_df.at[index,'P/L'] = Short_position_df.loc[index,'Entry_Price'] - Short_position_df.loc[index,'Exit_Price']

position = input("Enter Position(L/S)\n")
max_holding_period = int(input("Enter the maximum holding period of a position\n") or 28)

if(position == 'L'):
    target_p = float(input("Enter the target(in %) wrt Entry Price\n") or 7)
    stoploss_p = float(input("Enter the stoploss(in %) wrt Entry Price\n") or 2.5)
    Long_position_df = pd.DataFrame(columns = ['Entry_Date','Entry_Price','Exit_Date','Exit_Price','StopLoss','Target','Holding_Period','P/L'])
    
    #todo: provide user a list of patterns to select from
    filt = ((pattern_signal_df['Bull_Maru'] == 1) |
            (pattern_signal_df['Hammer'] == 1) |
            (pattern_signal_df['Bull_Engulf'] == 1) |
            (pattern_signal_df['Bull_Harami'] == 1) |
            (pattern_signal_df['Gap_Up'] == 1) |
           (pattern_signal_df['Morning_Star'] == 1))
    temp_signal_df = pattern_signal_df[filt]
    
    index = 0
    for i in range(stock_data.shape[0]):
        if(stock_data.index[i] in temp_signal_df.index):
            if((i+max_holding_period+1)<stock_data.shape[0]):
                Long_position_df.at[index,'Entry_Date'] = stock_data.index[i+1]
                Long_position_df.at[index,'Entry_Price'] = stock_data.loc[stock_data.index[i+1],'Open']
                long_trade(i)
                index += 1
    display(Long_position_df)
    print("Total Trades\t"+str(Long_position_df.shape[0]))
    print("Profitable Trades\t"+str(Long_position_df[(Long_position_df['Exit_Price'] > Long_position_df['Entry_Price'])].shape[0]))
    print("Non-Profitable Trades\t"+str(Long_position_df[(Long_position_df['Exit_Price'] <= Long_position_df['Entry_Price'])].shape[0]))
    print("Total Investment\t"+str(Long_position_df['Entry_Price'].sum()))
    print("Current Value\t"+str(Long_position_df['Exit_Price'].sum()))
    print("Net P/L\t"+str(Long_position_df['P/L'].sum()))
    
elif(position == 'S'):
    target_p = float(input("Enter the target(in %) wrt Entry Price\n") or 7)
    stoploss_p = float(input("Enter the stoploss(in %) wrt Entry Price\n") or 2.5)
    Short_position_df = pd.DataFrame(columns = ['Entry_Date','Entry_Price','Exit_Date','Exit_Price','StopLoss','Target','Holding_Period','P/L'])
    
    #todo: provide user a list of patterns to select from
    filt = (#(pattern_signal_df['Bear_Maru'] == 1) |
            (pattern_signal_df['Shooting_Star'] == 1) |
            (pattern_signal_df['Bear_Engulf'] == 1) |
            (pattern_signal_df['Bear_Harami'] == 1) |
            (pattern_signal_df['Gap_Down'] == 1) |
           (pattern_signal_df['Evening_Star'] == 1))
            
    temp_signal_df = pattern_signal_df[filt]
    
    index = 0
    for i in range(stock_data.shape[0]):
        if(stock_data.index[i] in temp_signal_df.index):
            if((i+max_holding_period+1)<stock_data.shape[0]):
                Short_position_df.at[index,'Entry_Date'] = stock_data.index[i+1]
                Short_position_df.at[index,'Entry_Price'] = stock_data.loc[stock_data.index[i+1],'Open']
                Short_trade(i)
                index += 1
    display(Short_position_df)
    print("Total Trades\t"+str(Short_position_df.shape[0]))
    print("Profitable Trades\t"+str(Short_position_df[(Short_position_df['Exit_Price'] < Short_position_df['Entry_Price'])].shape[0]))
    print("Non-Profitable Trades\t"+str(Short_position_df[(Short_position_df['Exit_Price'] >= Short_position_df['Entry_Price'])].shape[0]))
    print("Total Investment\t"+str(Short_position_df['Entry_Price'].sum()))
    print("Current Value\t"+str(Short_position_df['Exit_Price'].sum()))
    print("Net P/L\t"+str(Short_position_df['P/L'].sum()))
    

Failed. look in trade freq. for HUl, trade freq of 1 is high, and parameter used there are 5% target and 2.5% stoploss
so with these values the return for HUL(5% long with 2.5% sl or 6% with 3% sl for 30 day) is also high.

STRATERGY 2: Backtesting a candlestick pattern with Indicators:

In [ ]:
def long_indicator_confirmation(i):
    posi = Long_Indicator_confirmation.shape[0]
    Long_Indicator_confirmation.at[posi,'Date'] = stock_data.index[i]
    if(stock_data.index[i] in stock_data_indicator_df.index):
        #Volume Confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'Volume'] > stock_data_indicator_df.loc[stock_data.index[i],'Volume_SMA10'])):
            Long_Indicator_confirmation.at[posi,'Volume'] = 1
        else:
            Long_Indicator_confirmation.at[posi,'Volume'] = 0
        #Moving Average confirmation

        #RSI confirmation

        #MACD confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'MACD12_26'] > stock_data_indicator_df.loc[stock_data.index[i],'MACD_Signal_9'])):
            Long_Indicator_confirmation.at[posi,'MACD'] = 1
        else:
            Long_Indicator_confirmation.at[posi,'MACD'] = 0
        #ADX confirmation
        if(stock_data_indicator_df.loc[stock_data.index[i],'ADX'] > 25):
            if((stock_data_indicator_df.loc[stock_data.index[i],'plus_DI'] > stock_data_indicator_df.loc[stock_data.index[i],'minus_DI'])):
                Long_Indicator_confirmation.at[posi,'ADX'] = 1
            else:
                Long_Indicator_confirmation.at[posi,'ADX'] = 0

        #PSAR Trend confirmation

        if(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == 1):
            Long_Indicator_confirmation.at[posi,'PSAR_Trend'] = 1
        else:
            Long_Indicator_confirmation.at[posi,'PSAR_Trend'] = 0

        Indicators_confirmed = 0
        for column in Long_Indicator_confirmation.columns[1:]:
            if(Long_Indicator_confirmation.loc[posi,column] == 1):
                Indicators_confirmed += 1
        if(Indicators_confirmed >= 1):
            return 1
        
        
def short_indicator_confirmation(i):
    posi = Short_Indicator_confirmation.shape[0]
    Short_Indicator_confirmation.at[posi,'Date'] = stock_data.index[i]
    if(stock_data.index[i] in stock_data_indicator_df.index):
        #Volume Confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'Volume'] > stock_data_indicator_df.loc[stock_data.index[i],'Volume_SMA10'])):
            Short_Indicator_confirmation.at[posi,'Volume'] = 1
        else:
            Short_Indicator_confirmation.at[posi,'Volume'] = 0
        #Moving Average confirmation

        #RSI confirmation

        #MACD confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'MACD12_26'] < stock_data_indicator_df.loc[stock_data.index[i],'MACD_Signal_9'])):
            Short_Indicator_confirmation.at[posi,'MACD'] = 1
        else:
            Short_Indicator_confirmation.at[posi,'MACD'] = 0
        #ADX confirmation
        if(stock_data_indicator_df.loc[stock_data.index[i],'ADX'] > 25):
            if((stock_data_indicator_df.loc[stock_data.index[i],'plus_DI'] < stock_data_indicator_df.loc[stock_data.index[i],'minus_DI'])):
                Short_Indicator_confirmation.at[posi,'ADX'] = 1
            else:
                Short_Indicator_confirmation.at[posi,'ADX'] = 0

        #PSAR Trend confirmation

        if(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == -1):
            Short_Indicator_confirmation.at[posi,'PSAR_Trend'] = 1
        else:
            Short_Indicator_confirmation.at[posi,'PSAR_Trend'] = 0

        Indicators_confirmed = 0
        for column in Short_Indicator_confirmation.columns[1:]:
            if(Short_Indicator_confirmation.loc[posi,column] == 1):
                Indicators_confirmed += 1
        if(Indicators_confirmed >= 4):
            return 1
        
                
def long_trade(j):
    i = j+1
    price = stock_data.loc[stock_data.index[i],'Open']
    target = price * (1+(target_p/100))
    stoploss = price * (1-(stoploss_p/100))
    Long_position_df.at[index,'StopLoss'] = stoploss
    Long_position_df.at[index,'Target'] = target
    max_period = i+max_holding_period
    while(price < target and price > stoploss and i<max_period):
        i += 1
        if(stock_data.loc[stock_data.index[i],'High'] > target):
            Long_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Long_position_df.at[index,'Exit_Price'] = target
            Long_position_df.at[index,'Holding_Period'] = (i - (max_period-max_holding_period))
            price = target
        elif(stock_data.loc[stock_data.index[i],'Low'] < stoploss):
            Long_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Long_position_df.at[index,'Exit_Price'] = stoploss
            Long_position_df.at[index,'Holding_Period'] = (i - (max_period-max_holding_period))
            price = stoploss
        else:
            Long_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Long_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i],'Close']
            Long_position_df.at[index,'Holding_Period'] = (i - (max_period-max_holding_period))
            price = stock_data.loc[stock_data.index[i],'Close']
    Long_position_df.at[index,'P/L'] = Long_position_df.loc[index,'Exit_Price'] - Long_position_df.loc[index,'Entry_Price']

def Short_trade(j):
    i = j+1
    price = stock_data.loc[stock_data.index[i],'Open']
    target = price * (1-(target_p/100))
    stoploss = price * (1+(stoploss_p/100))
    Short_position_df.at[index,'StopLoss'] = stoploss
    Short_position_df.at[index,'Target'] = target
    max_period = i+max_holding_period
    while(price > target and price < stoploss and i<max_period):
        i += 1
        if(stock_data.loc[stock_data.index[i],'High'] > stoploss):
            Short_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Short_position_df.at[index,'Exit_Price'] = stoploss
            Short_position_df.at[index,'Holding_Period'] = (i - (max_period-max_holding_period))
            price = stoploss
        elif(stock_data.loc[stock_data.index[i],'Low'] < target):
            Short_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Short_position_df.at[index,'Exit_Price'] = target
            Short_position_df.at[index,'Holding_Period'] = (i - (max_period-max_holding_period))
            price = target
        else:
            Short_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Short_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i],'Close']
            Short_position_df.at[index,'Holding_Period'] = (i - (max_period-max_holding_period))
            price = stock_data.loc[stock_data.index[i],'Close']
    Short_position_df.at[index,'P/L'] = Short_position_df.loc[index,'Entry_Price'] - Short_position_df.loc[index,'Exit_Price']

    
    
position = input("Enter Position(L/S)\n")
max_holding_period = int(input("Enter the maximum holding period of a position\n") or 28)

if(position == 'L'):
    target_p = float(input("Enter the target(in %) wrt Entry Price\n") or 7)
    stoploss_p = float(input("Enter the stoploss(in %) wrt Entry Price\n") or 2.5)
    Long_position_df = pd.DataFrame(columns = ['Entry_Date','Entry_Price','Exit_Date','Exit_Price','StopLoss','Target','Holding_Period','P/L'])
    
    #creating a Long position Indicator confirmation df:
    Long_Indicator_confirmation = pd.DataFrame(columns=['Date','Volume','Moving_Average','RSI','MACD','ADX','PSAR_trend'])
    
    #todo: provide user a list of patterns to select from
    filt = ((pattern_signal_df['Bull_Maru'] == 1) |
            (pattern_signal_df['Hammer'] == 1) |
            (pattern_signal_df['Bull_Engulf'] == 1) |
            (pattern_signal_df['Bull_Harami'] == 1) |
            (pattern_signal_df['Gap_Up'] == 1) |
           (pattern_signal_df['Morning_Star'] == 1))
    temp_signal_df = pattern_signal_df[filt]
    
    index = 0
    for i in range(stock_data.shape[0]):
        if(stock_data.index[i] in temp_signal_df.index):
            if((i+max_holding_period+1)<stock_data.shape[0]):
                indicator_confirmation = long_indicator_confirmation(i)                    
                if(indicator_confirmation == 1):
                    Long_position_df.at[index,'Entry_Date'] = stock_data.index[i+1]
                    Long_position_df.at[index,'Entry_Price'] = stock_data.loc[stock_data.index[i+1],'Open']
                    long_trade(i)
                    index += 1
    display(Long_position_df)
    print("Total Trades\t"+str(Long_position_df.shape[0]))
    print("Profitable Trades\t"+str(Long_position_df[(Long_position_df['Exit_Price'] > Long_position_df['Entry_Price'])].shape[0]))
    print("Non-Profitable Trades\t"+str(Long_position_df[(Long_position_df['Exit_Price'] <= Long_position_df['Entry_Price'])].shape[0]))
    print("Total Investment\t"+str(Long_position_df['Entry_Price'].sum()))
    print("Current Value\t"+str(Long_position_df['Exit_Price'].sum()))
    print("Net P/L\t"+str(Long_position_df['P/L'].sum()))
    
elif(position == 'S'):
    target_p = float(input("Enter the target(in %) wrt Entry Price\n") or 7)
    stoploss_p = float(input("Enter the stoploss(in %) wrt Entry Price\n") or 2.5)
    Short_position_df = pd.DataFrame(columns = ['Entry_Date','Entry_Price','Exit_Date','Exit_Price','StopLoss','Target','Holding_Period','P/L'])
    
    #creating a short position Indicator confirmation df:
    Short_Indicator_confirmation = pd.DataFrame(columns=['Date','Volume','Moving_Average','RSI','MACD','ADX','PSAR_trend'])
    
    #todo: provide user a list of patterns to select from
    filt = ((pattern_signal_df['Bear_Maru'] == 1) |
            (pattern_signal_df['Shooting_Star'] == 1) |
            (pattern_signal_df['Bear_Engulf'] == 1) |
            (pattern_signal_df['Bear_Harami'] == 1) |
            (pattern_signal_df['Gap_Down'] == 1) |
           (pattern_signal_df['Evening_Star'] == 1))
            
    temp_signal_df = pattern_signal_df[filt]
    
    index = 0
    for i in range(stock_data.shape[0]):
        if(stock_data.index[i] in temp_signal_df.index):
            if((i+max_holding_period+1)<stock_data.shape[0]):
                indicator_confirmation = short_indicator_confirmation(i)
                if(indicator_confirmation == 1):
                    Short_position_df.at[index,'Entry_Date'] = stock_data.index[i+1]
                    Short_position_df.at[index,'Entry_Price'] = stock_data.loc[stock_data.index[i+1],'Open']
                    Short_trade(i)
                    index += 1
    display(Short_position_df)
    print("Total Trades\t"+str(Short_position_df.shape[0]))
    print("Profitable Trades\t"+str(Short_position_df[(Short_position_df['Exit_Price'] < Short_position_df['Entry_Price'])].shape[0]))
    print("Non-Profitable Trades\t"+str(Short_position_df[(Short_position_df['Exit_Price'] >= Short_position_df['Entry_Price'])].shape[0]))
    print("Total Investment\t"+str(Short_position_df['Entry_Price'].sum()))
    print("Current Value\t"+str(Short_position_df['Exit_Price'].sum()))
    print("Net P/L\t"+str(Short_position_df['P/L'].sum()))
    

Failed for TCS. works for HUL.for HUL, the trend 1 is high.

STRATERGY 3: Using a PSAR to exit rather than fixed Target and Stoploss.

In [ ]:
def long_indicator_confirmation(i):
    posi = Long_Indicator_confirmation.shape[0]
    Long_Indicator_confirmation.at[posi,'Date'] = stock_data.index[i]
    if(stock_data.index[i] in stock_data_indicator_df.index):
        #Volume Confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'Volume'] > stock_data_indicator_df.loc[stock_data.index[i],'Volume_SMA10'])):
            Long_Indicator_confirmation.at[posi,'Volume'] = 1
        else:
            Long_Indicator_confirmation.at[posi,'Volume'] = 0
        #Moving Average confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'EMA7'] > stock_data_indicator_df.loc[stock_data.index[i],'EMA21'])):
            Long_Indicator_confirmation.at[posi,'MA'] = 1
        else:
            Long_Indicator_confirmation.at[posi,'MA'] = 0

        #RSI confirmation

        #MACD confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'MACD12_26'] > stock_data_indicator_df.loc[stock_data.index[i],'MACD_Signal_9'])):
            Long_Indicator_confirmation.at[posi,'MACD'] = 1
        else:
            Long_Indicator_confirmation.at[posi,'MACD'] = 0
        #ADX confirmation
        if(stock_data_indicator_df.loc[stock_data.index[i],'ADX'] > 25):
            if((stock_data_indicator_df.loc[stock_data.index[i],'plus_DI'] > stock_data_indicator_df.loc[stock_data.index[i],'minus_DI'])):
                Long_Indicator_confirmation.at[posi,'ADX'] = 1
            else:
                Long_Indicator_confirmation.at[posi,'ADX'] = 0

        #PSAR prior Trend confirmation

        if((stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == 1)&
          ((stock_data_indicator_df.loc[stock_data.index[i-1],'Trend'] == -1) | (stock_data_indicator_df.loc[stock_data.index[i-2],'Trend'] == -1))):
            Long_Indicator_confirmation.at[posi,'PSAR_Trend'] = 1
        else:
            Long_Indicator_confirmation.at[posi,'PSAR_Trend'] = 0

        Indicators_confirmed = 0
        for column in Long_Indicator_confirmation.columns[1:]:
            if(Long_Indicator_confirmation.loc[posi,column] == 1):
                Indicators_confirmed += 1
        if(Indicators_confirmed >= 4):
            return 1
        
        
def short_indicator_confirmation(i):
    posi = Short_Indicator_confirmation.shape[0]
    Short_Indicator_confirmation.at[posi,'Date'] = stock_data.index[i]
    if(stock_data.index[i] in stock_data_indicator_df.index):
        #Volume Confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'Volume'] > stock_data_indicator_df.loc[stock_data.index[i],'Volume_SMA10'])):
            Short_Indicator_confirmation.at[posi,'Volume'] = 1
        else:
            Short_Indicator_confirmation.at[posi,'Volume'] = 0
        #Moving Average confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'EMA7'] < stock_data_indicator_df.loc[stock_data.index[i],'EMA21'])):
            Short_Indicator_confirmation.at[posi,'MA'] = 1
        else:
            Short_Indicator_confirmation.at[posi,'MA'] = 0

        #RSI confirmation

        #MACD confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'MACD12_26'] < stock_data_indicator_df.loc[stock_data.index[i],'MACD_Signal_9'])):
            Short_Indicator_confirmation.at[posi,'MACD'] = 1
        else:
            Short_Indicator_confirmation.at[posi,'MACD'] = 0
        #ADX confirmation
        if(stock_data_indicator_df.loc[stock_data.index[i],'ADX'] > 25):
            if((stock_data_indicator_df.loc[stock_data.index[i],'plus_DI'] < stock_data_indicator_df.loc[stock_data.index[i],'minus_DI'])):
                Short_Indicator_confirmation.at[posi,'ADX'] = 1
            else:
                Short_Indicator_confirmation.at[posi,'ADX'] = 0

        #PSAR Trend confirmation

        if((stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == -1)&
          ((stock_data_indicator_df.loc[stock_data.index[i-1],'Trend'] == 1) | (stock_data_indicator_df.loc[stock_data.index[i-2],'Trend'] == 1))):
            Short_Indicator_confirmation.at[posi,'PSAR_Trend'] = 1
        else:
            Short_Indicator_confirmation.at[posi,'PSAR_Trend'] = 0

        Indicators_confirmed = 0
        for column in Short_Indicator_confirmation.columns[1:]:
            if(Short_Indicator_confirmation.loc[posi,column] == 1):
                Indicators_confirmed += 1
        if(Indicators_confirmed >= 4):
            return 1
        
                
def long_trade(j):
    i = j+1
    price = stock_data.loc[stock_data.index[i],'Open']
    if(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == 1):
        while(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] != -1 and i < (stock_data.shape[0]-1)):
            i += 1
            Long_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Long_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i],'Close']
            Long_position_df.at[index,'Holding_Period'] = (i-j-1)
        Long_position_df.at[index,'P/L'] = Long_position_df.loc[index,'Exit_Price'] - Long_position_df.loc[index,'Entry_Price']
    elif(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == -1):
        Long_position_df.at[index,'Exit_Date'] = stock_data.index[i]
        Long_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i],'Close']
        Long_position_df.at[index,'Holding_Period'] = (i-j-1)
        Long_position_df.at[index,'P/L'] = Long_position_df.loc[index,'Exit_Price'] - Long_position_df.loc[index,'Entry_Price']
        

def Short_trade(j):
    i = j+1
    price = stock_data.loc[stock_data.index[i],'Open']
    if(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == -1):
        while(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] != 1 and i < (stock_data.shape[0]-1)):
            i += 1
            Short_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Short_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i],'Close']
            Short_position_df.at[index,'Holding_Period'] = (i-j-1)       
        Short_position_df.at[index,'P/L'] = Short_position_df.loc[index,'Entry_Price'] - Short_position_df.loc[index,'Exit_Price']
    elif(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == 1):
        Short_position_df.at[index,'Exit_Date'] = stock_data.index[i]
        Short_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i],'Close']
        Short_position_df.at[index,'Holding_Period'] = (i-j-1)       
        Short_position_df.at[index,'P/L'] = Short_position_df.loc[index,'Entry_Price'] - Short_position_df.loc[index,'Exit_Price']
    
position = input("Enter Position(L/S)\n")

if(position == 'L'):
    Long_position_df = pd.DataFrame(columns = ['Entry_Date','Entry_Price','Exit_Date','Exit_Price','Holding_Period','P/L'])
    
    #creating a Long position Indicator confirmation df:
    Long_Indicator_confirmation = pd.DataFrame(columns=['Date','Volume','MA','RSI','MACD','ADX','PSAR_trend'])
    
    #todo: provide user a list of patterns to select from
    filt = ((pattern_signal_df['Bull_Maru'] == 1) |
            (pattern_signal_df['Hammer'] == 1) |
            (pattern_signal_df['Bull_Engulf'] == 1) |
            (pattern_signal_df['Bull_Harami'] == 1) |
            (pattern_signal_df['Gap_Up'] == 1) |
           (pattern_signal_df['Morning_Star'] == 1))
    temp_signal_df = pattern_signal_df[filt]
    
    index = 0
    for i in range(stock_data.shape[0]):
        if(stock_data.index[i] in temp_signal_df.index):
            if(i<stock_data.shape[0]-2):
                indicator_confirmation = long_indicator_confirmation(i)                    
                if(indicator_confirmation == 1):
                    Long_position_df.at[index,'Entry_Date'] = stock_data.index[i+1]
                    Long_position_df.at[index,'Entry_Price'] = stock_data.loc[stock_data.index[i+1],'Open']
                    long_trade(i)
                    index += 1
    display(Long_position_df)
    print("Total Trades\t"+str(Long_position_df.shape[0]))
    print("Profitable Trades\t"+str(Long_position_df[(Long_position_df['Exit_Price'] > Long_position_df['Entry_Price'])].shape[0]))
    print("Non-Profitable Trades\t"+str(Long_position_df[(Long_position_df['Exit_Price'] <= Long_position_df['Entry_Price'])].shape[0]))
    print("Total Investment\t"+str(Long_position_df['Entry_Price'].sum()))
    print("Current Value\t"+str(Long_position_df['Exit_Price'].sum()))
    print("Net P/L\t"+str(Long_position_df['P/L'].sum()))
    
elif(position == 'S'):
    Short_position_df = pd.DataFrame(columns = ['Entry_Date','Entry_Price','Exit_Date','Exit_Price','Holding_Period','P/L'])
    
    #creating a short position Indicator confirmation df:
    Short_Indicator_confirmation = pd.DataFrame(columns=['Date','Volume','MA','RSI','MACD','ADX','PSAR_trend'])
    
    #todo: provide user a list of patterns to select from
    filt = ((pattern_signal_df['Bear_Maru'] == 1) |
            (pattern_signal_df['Shooting_Star'] == 1) |
            (pattern_signal_df['Bear_Engulf'] == 1) |
            (pattern_signal_df['Bear_Harami'] == 1) |
            (pattern_signal_df['Gap_Down'] == 1) |
           (pattern_signal_df['Evening_Star'] == 1))
            
    temp_signal_df = pattern_signal_df[filt]
    
    index = 0
    for i in range(stock_data.shape[0]):
        if(stock_data.index[i] in temp_signal_df.index):
            if((i+1)<stock_data.shape[0]):
                indicator_confirmation = short_indicator_confirmation(i)
                if(indicator_confirmation == 1):
                    Short_position_df.at[index,'Entry_Date'] = stock_data.index[i+1]
                    Short_position_df.at[index,'Entry_Price'] = stock_data.loc[stock_data.index[i+1],'Open']
                    Short_trade(i)
                    index += 1
    display(Short_position_df)
    print("Total Trades\t"+str(Short_position_df.shape[0]))
    print("Profitable Trades\t"+str(Short_position_df[(Short_position_df['Exit_Price'] < Short_position_df['Entry_Price'])].shape[0]))
    print("Non-Profitable Trades\t"+str(Short_position_df[(Short_position_df['Exit_Price'] >= Short_position_df['Entry_Price'])].shape[0]))
    print("Total Investment\t"+str(Short_position_df['Entry_Price'].sum()))
    print("Current Value\t"+str(Short_position_df['Exit_Price'].sum()))
    print("Net P/L\t"+str(Short_position_df['P/L'].sum()))
    

Failed.
The Problem here is with whipsaws wherein the Holding Period are 0 days.
To overcome this, we need to identify trending stocks and enter only around the start of a reversal and not near the end.

STRATERGY 4: stratergy 3 modified- entry based on PSAR reversal

In [ ]:
def long_indicator_confirmation(i):
    posi = Long_Indicator_confirmation.shape[0]
    Long_Indicator_confirmation.at[posi,'Date'] = stock_data.index[i]
    if(stock_data.index[i] in stock_data_indicator_df.index):
        #Volume Confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'Volume'] > stock_data_indicator_df.loc[stock_data.index[i],'Volume_SMA10'])):
            Long_Indicator_confirmation.at[posi,'Volume'] = 1
        else:
            Long_Indicator_confirmation.at[posi,'Volume'] = 0
        #Moving Average confirmation

        #RSI confirmation

        #MACD confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'MACD12_26'] > stock_data_indicator_df.loc[stock_data.index[i],'MACD_Signal_9'])):
            Long_Indicator_confirmation.at[posi,'MACD'] = 1
        else:
            Long_Indicator_confirmation.at[posi,'MACD'] = 0
        #ADX confirmation
        if(stock_data_indicator_df.loc[stock_data.index[i],'ADX'] > 25):
            if((stock_data_indicator_df.loc[stock_data.index[i],'plus_DI'] > stock_data_indicator_df.loc[stock_data.index[i],'minus_DI'])):
                Long_Indicator_confirmation.at[posi,'ADX'] = 1
            else:
                Long_Indicator_confirmation.at[posi,'ADX'] = 0

        #PSAR prior Trend confirmation

        if(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == 1):
            Long_Indicator_confirmation.at[posi,'PSAR_Trend'] = 1
        else:
            Long_Indicator_confirmation.at[posi,'PSAR_Trend'] = 0

        Indicators_confirmed = 0
        for column in Long_Indicator_confirmation.columns[1:]:
            if(Long_Indicator_confirmation.loc[posi,column] == 1):
                Indicators_confirmed += 1
        if(Indicators_confirmed >= 4):
            return 1
        
        
def short_indicator_confirmation(i):
    posi = Short_Indicator_confirmation.shape[0]
    Short_Indicator_confirmation.at[posi,'Date'] = stock_data.index[i]
    if(stock_data.index[i] in stock_data_indicator_df.index):
        #Volume Confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'Volume'] > stock_data_indicator_df.loc[stock_data.index[i],'Volume_SMA10'])):
            Short_Indicator_confirmation.at[posi,'Volume'] = 1
        else:
            Short_Indicator_confirmation.at[posi,'Volume'] = 0
        #Moving Average confirmation

        #RSI confirmation

        #MACD confirmation
        if((stock_data_indicator_df.loc[stock_data.index[i],'MACD12_26'] < stock_data_indicator_df.loc[stock_data.index[i],'MACD_Signal_9'])):
            Short_Indicator_confirmation.at[posi,'MACD'] = 1
        else:
            Short_Indicator_confirmation.at[posi,'MACD'] = 0
        #ADX confirmation
        if(stock_data_indicator_df.loc[stock_data.index[i],'ADX'] > 25):
            if((stock_data_indicator_df.loc[stock_data.index[i],'plus_DI'] < stock_data_indicator_df.loc[stock_data.index[i],'minus_DI'])):
                Short_Indicator_confirmation.at[posi,'ADX'] = 1
            else:
                Short_Indicator_confirmation.at[posi,'ADX'] = 0

        #PSAR Trend confirmation

        if(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == -1):
            Long_Indicator_confirmation.at[posi,'PSAR_Trend'] = 1
        else:
            Long_Indicator_confirmation.at[posi,'PSAR_Trend'] = 0

        Indicators_confirmed = 0
        for column in Long_Indicator_confirmation.columns[1:]:
            if(Long_Indicator_confirmation.loc[posi,column] == 1):
                Indicators_confirmed += 1
        if(Indicators_confirmed >= 4):
            return 1
        
                
def long_trade(j):
    i = j+1
    price = stock_data.loc[stock_data.index[i],'Open']
    if(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == 1):
        while(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] != -1 and i < (stock_data.shape[0]-1)):
            i += 1
            Long_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Long_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i],'Close']
            Long_position_df.at[index,'Holding_Period'] = (i-j-1)
        Long_position_df.at[index,'P/L'] = Long_position_df.loc[index,'Exit_Price'] - Long_position_df.loc[index,'Entry_Price']
    elif(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == -1):
        Long_position_df.at[index,'Exit_Date'] = stock_data.index[i]
        Long_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i],'Close']
        Long_position_df.at[index,'Holding_Period'] = (i-j-1)
        Long_position_df.at[index,'P/L'] = Long_position_df.loc[index,'Exit_Price'] - Long_position_df.loc[index,'Entry_Price']
        

def Short_trade(j):
    i = j+1
    price = stock_data.loc[stock_data.index[i],'Open']
    if(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == -1):
        while(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] != 1 and i < (stock_data.shape[0]-1)):
            i += 1
            Short_position_df.at[index,'Exit_Date'] = stock_data.index[i]
            Short_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i],'Close']
            Short_position_df.at[index,'Holding_Period'] = (i-j-1)       
        Short_position_df.at[index,'P/L'] = Short_position_df.loc[index,'Entry_Price'] - Short_position_df.loc[index,'Exit_Price']
    elif(stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == 1):
        Short_position_df.at[index,'Exit_Date'] = stock_data.index[i]
        Short_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i],'Close']
        Short_position_df.at[index,'Holding_Period'] = (i-j-1)       
        Short_position_df.at[index,'P/L'] = Short_position_df.loc[index,'Entry_Price'] - Short_position_df.loc[index,'Exit_Price']
    
position = input("Enter Position(L/S)\n")

if(position == 'L'):
    Long_position_df = pd.DataFrame(columns = ['Entry_Date','Entry_Price','Exit_Date','Exit_Price','Holding_Period','P/L'])
    
    #creating a Long position Indicator confirmation df:
    Long_Indicator_confirmation = pd.DataFrame(columns=['Date','Volume','Moving_Average','RSI','MACD','ADX','PSAR_trend'])
    
    #todo: provide user a list of patterns to select from
    filt = ((pattern_signal_df['Bull_Maru'] == 1) |
            (pattern_signal_df['Hammer'] == 1) |
            (pattern_signal_df['Bull_Engulf'] == 1) |
            (pattern_signal_df['Bull_Harami'] == 1) |
            (pattern_signal_df['Gap_Up'] == 1) |
           (pattern_signal_df['Morning_Star'] == 1))
    temp_signal_df = pattern_signal_df[filt]
    
    index = 0
    for i in range(stock_data.shape[0]):
        if(stock_data.index[i] in temp_signal_df.index):
            if(i<stock_data.shape[0]-2):
                indicator_confirmation = long_indicator_confirmation(i)                    
                if(indicator_confirmation == 1):
                    Long_position_df.at[index,'Entry_Date'] = stock_data.index[i+1]
                    Long_position_df.at[index,'Entry_Price'] = stock_data.loc[stock_data.index[i+1],'Open']
                    long_trade(i)
                    index += 1
    display(Long_position_df)
    print("Total Trades\t"+str(Long_position_df.shape[0]))
    print("Profitable Trades\t"+str(Long_position_df[(Long_position_df['Exit_Price'] > Long_position_df['Entry_Price'])].shape[0]))
    print("Non-Profitable Trades\t"+str(Long_position_df[(Long_position_df['Exit_Price'] <= Long_position_df['Entry_Price'])].shape[0]))
    print("Total Investment\t"+str(Long_position_df['Entry_Price'].sum()))
    print("Current Value\t"+str(Long_position_df['Exit_Price'].sum()))
    print("Net P/L\t"+str(Long_position_df['P/L'].sum()))
    
elif(position == 'S'):
    Short_position_df = pd.DataFrame(columns = ['Entry_Date','Entry_Price','Exit_Date','Exit_Price','Holding_Period','P/L'])
    
    #creating a short position Indicator confirmation df:
    Short_Indicator_confirmation = pd.DataFrame(columns=['Date','Volume','Moving_Average','RSI','MACD','ADX','PSAR_trend'])
    
    #todo: provide user a list of patterns to select from
    filt = ((pattern_signal_df['Bear_Maru'] == 1) |
            (pattern_signal_df['Shooting_Star'] == 1) |
            (pattern_signal_df['Bear_Engulf'] == 1) |
            (pattern_signal_df['Bear_Harami'] == 1) |
            (pattern_signal_df['Gap_Down'] == 1) |
           (pattern_signal_df['Evening_Star'] == 1))
            
    temp_signal_df = pattern_signal_df[filt]
    
    index = 0
    for i in range(stock_data.shape[0]):
        if(stock_data.index[i] in temp_signal_df.index):
            if((i+1)<stock_data.shape[0]):
                indicator_confirmation = short_indicator_confirmation(i)
                if(indicator_confirmation == 1):
                    Short_position_df.at[index,'Entry_Date'] = stock_data.index[i+1]
                    Short_position_df.at[index,'Entry_Price'] = stock_data.loc[stock_data.index[i+1],'Open']
                    Short_trade(i)
                    index += 1
    display(Short_position_df)
    print("Total Trades\t"+str(Short_position_df.shape[0]))
    print("Profitable Trades\t"+str(Short_position_df[(Short_position_df['Exit_Price'] < Short_position_df['Entry_Price'])].shape[0]))
    print("Non-Profitable Trades\t"+str(Short_position_df[(Short_position_df['Exit_Price'] >= Short_position_df['Entry_Price'])].shape[0]))
    print("Total Investment\t"+str(Short_position_df['Entry_Price'].sum()))
    print("Current Value\t"+str(Short_position_df['Exit_Price'].sum()))
    print("Net P/L\t"+str(Short_position_df['P/L'].sum()))
    

Failed

STRATERGY 5: Testing the candlestick pattern for intraday trades

In [ ]:
position = input("Enter Position(L/S)\n")

if(position == 'L'):
    Long_position_df = pd.DataFrame(columns = ['Date','Entry_Price','Exit_Price','P/L'])
    
    #todo: provide user a list of patterns to select from
    filt = ((pattern_signal_df['Bull_Maru'] == 1) |
            (pattern_signal_df['Hammer'] == 1) |
            (pattern_signal_df['Bull_Engulf'] == 1) |
            (pattern_signal_df['Bull_Harami'] == 1) |
            (pattern_signal_df['Gap_Up'] == 1) |
           (pattern_signal_df['Morning_Star'] == 1))
    temp_signal_df = pattern_signal_df[filt]
    
    index = 0
    for i in range(stock_data.shape[0]):
        if(stock_data.index[i] in temp_signal_df.index):
            if(i<stock_data.shape[0]-1):
                Long_position_df.at[index,'Date'] = stock_data.index[i+1]
                Long_position_df.at[index,'Entry_Price'] = stock_data.loc[stock_data.index[i+1],'Open']
                Long_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i+1],'Close']
                Long_position_df.at[index,'P/L'] = Long_position_df.loc[index,'Exit_Price'] - Long_position_df.loc[index,'Entry_Price']
                index += 1
    display(Long_position_df)
    print("Total Trades\t"+str(Long_position_df.shape[0]))
    print("Profitable Trades\t"+str(Long_position_df[(Long_position_df['Exit_Price'] > Long_position_df['Entry_Price'])].shape[0]))
    print("Non-Profitable Trades\t"+str(Long_position_df[(Long_position_df['Exit_Price'] <= Long_position_df['Entry_Price'])].shape[0]))
    print("Total Investment\t"+str(Long_position_df['Entry_Price'].sum()))
    print("Current Value\t"+str(Long_position_df['Exit_Price'].sum()))
    print("Net P/L\t"+str(Long_position_df['P/L'].sum()))
    
elif(position == 'S'):
    Short_position_df = pd.DataFrame(columns = ['Date','Entry_Price','Exit_Price','P/L'])
    
    #todo: provide user a list of patterns to select from
    filt = ((pattern_signal_df['Bear_Maru'] == 1) |
            (pattern_signal_df['Shooting_Star'] == 1) |
            (pattern_signal_df['Bear_Engulf'] == 1) |
            (pattern_signal_df['Bear_Harami'] == 1) |
            (pattern_signal_df['Gap_Down'] == 1) |
           (pattern_signal_df['Evening_Star'] == 1))
            
    temp_signal_df = pattern_signal_df[filt]
    
    index = 0
    for i in range(stock_data.shape[0]):
        if(stock_data.index[i] in temp_signal_df.index):
            if(i<stock_data.shape[0]-1):
                Short_position_df.at[index,'Date'] = stock_data.index[i+1]
                Short_position_df.at[index,'Entry_Price'] = stock_data.loc[stock_data.index[i+1],'Open']
                Short_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i+1],'Close']
                Short_position_df.at[index,'P/L'] = Short_position_df.loc[index,'Entry_Price'] - Short_position_df.loc[index,'Exit_Price']
                index += 1
    display(Short_position_df)
    print("Total Trades\t"+str(Short_position_df.shape[0]))
    print("Profitable Trades\t"+str(Short_position_df[(Short_position_df['Exit_Price'] < Short_position_df['Entry_Price'])].shape[0]))
    print("Non-Profitable Trades\t"+str(Short_position_df[(Short_position_df['Exit_Price'] >= Short_position_df['Entry_Price'])].shape[0]))
    print("Total Investment\t"+str(Short_position_df['Entry_Price'].sum()))
    print("Current Value\t"+str(Short_position_df['Exit_Price'].sum()))
    print("Net P/L\t"+str(Short_position_df['P/L'].sum()))
    

Intraday following a candle pattern does not work. seemed to work for HDFC

STRATERGY 6 : Intraday trade using candlestick pattern and indicators

In [ ]:
def long_indicator_confirmation(j):
    i = j+1
    if(stock_data.index[i] in stock_data_indicator_df.index):
        if((stock_data_indicator_df.loc[stock_data.index[i],'Volume'] > stock_data_indicator_df.loc[stock_data.index[i],'Volume_SMA10']) &
          (stock_data_indicator_df.loc[stock_data.index[i],'plus_DI'] > stock_data_indicator_df.loc[stock_data.index[i],'minus_DI']) &
          (stock_data_indicator_df.loc[stock_data.index[i],'ADX'] >25) &
          (stock_data_indicator_df.loc[stock_data.index[i],'MACD12_26'] > stock_data_indicator_df.loc[stock_data.index[i],'MACD_Signal_9'])&
          (stock_data_indicator_df.loc[stock_data.index[i],'Trend'] ==1)):
            return 1
    
    
def short_indicator_confirmation(j):
    i = j+1
    if(stock_data.index[i] in stock_data_indicator_df.index):
        if((stock_data_indicator_df.loc[stock_data.index[i],'Volume'] > stock_data_indicator_df.loc[stock_data.index[i],'Volume_SMA10']) &
          (stock_data_indicator_df.loc[stock_data.index[i],'plus_DI'] < stock_data_indicator_df.loc[stock_data.index[i],'minus_DI']) &
          (stock_data_indicator_df.loc[stock_data.index[i],'ADX'] > 25) &
          (stock_data_indicator_df.loc[stock_data.index[i],'MACD12_26'] < stock_data_indicator_df.loc[stock_data.index[i],'MACD_Signal_9']) &
          (stock_data_indicator_df.loc[stock_data.index[i],'Trend'] == -1)):
            return 1
    
    
position = input("Enter Position(L/S)\n")

if(position == 'L'):
    Long_position_df = pd.DataFrame(columns = ['Date','Entry_Price','Exit_Price','P/L'])
    
    #todo: provide user a list of patterns to select from
    filt = ((pattern_signal_df['Bull_Maru'] == 1) |
            (pattern_signal_df['Hammer'] == 1) |
            (pattern_signal_df['Bull_Engulf'] == 1) |
            (pattern_signal_df['Bull_Harami'] == 1) |
            (pattern_signal_df['Gap_Up'] == 1) |
           (pattern_signal_df['Morning_Star'] == 1))
    temp_signal_df = pattern_signal_df[filt]
    
    index = 0
    for i in range(stock_data.shape[0]):
        if(stock_data.index[i] in temp_signal_df.index):
            if(i<stock_data.shape[0]-1):
                indicator_confirmation = long_indicator_confirmation(i)
                if(indicator_confirmation == 1):
                    Long_position_df.at[index,'Date'] = stock_data.index[i+1]
                    Long_position_df.at[index,'Entry_Price'] = stock_data.loc[stock_data.index[i+1],'Open']
                    Long_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i+1],'Close']
                    Long_position_df.at[index,'P/L'] = Long_position_df.loc[index,'Exit_Price'] - Long_position_df.loc[index,'Entry_Price']
                    index += 1
    display(Long_position_df)
    print("Total Trades\t"+str(Long_position_df.shape[0]))
    print("Profitable Trades\t"+str(Long_position_df[(Long_position_df['Exit_Price'] > Long_position_df['Entry_Price'])].shape[0]))
    print("Non-Profitable Trades\t"+str(Long_position_df[(Long_position_df['Exit_Price'] <= Long_position_df['Entry_Price'])].shape[0]))
    print("Total Investment\t"+str(Long_position_df['Entry_Price'].sum()))
    print("Current Value\t"+str(Long_position_df['Exit_Price'].sum()))
    print("Net P/L\t"+str(Long_position_df['P/L'].sum()))
    
elif(position == 'S'):
    Short_position_df = pd.DataFrame(columns = ['Date','Entry_Price','Exit_Price','P/L'])
    
    #todo: provide user a list of patterns to select from
    filt = ((pattern_signal_df['Bear_Maru'] == 1) |
            (pattern_signal_df['Shooting_Star'] == 1) |
            (pattern_signal_df['Bear_Engulf'] == 1) |
            (pattern_signal_df['Bear_Harami'] == 1) |
            (pattern_signal_df['Gap_Down'] == 1) |
           (pattern_signal_df['Evening_Star'] == 1))
            
    temp_signal_df = pattern_signal_df[filt]
    
    index = 0
    for i in range(stock_data.shape[0]):
        if(stock_data.index[i] in temp_signal_df.index):
            if(i<stock_data.shape[0]-1):
                indicator_confirmation = short_indicator_confirmation(i)
                if(indicator_confirmation == 1):
                    Short_position_df.at[index,'Date'] = stock_data.index[i+1]
                    Short_position_df.at[index,'Entry_Price'] = stock_data.loc[stock_data.index[i+1],'Open']
                    Short_position_df.at[index,'Exit_Price'] = stock_data.loc[stock_data.index[i+1],'Close']
                    Short_position_df.at[index,'P/L'] = Short_position_df.loc[index,'Entry_Price'] - Short_position_df.loc[index,'Exit_Price']
                    index += 1
    display(Short_position_df)
    print("Total Trades\t"+str(Short_position_df.shape[0]))
    print("Profitable Trades\t"+str(Short_position_df[(Short_position_df['Exit_Price'] < Short_position_df['Entry_Price'])].shape[0]))
    print("Non-Profitable Trades\t"+str(Short_position_df[(Short_position_df['Exit_Price'] >= Short_position_df['Entry_Price'])].shape[0]))
    print("Total Investment\t"+str(Short_position_df['Entry_Price'].sum()))
    print("Current Value\t"+str(Short_position_df['Exit_Price'].sum()))
    print("Net P/L\t"+str(Short_position_df['P/L'].sum()))
    

Failed. Need to modify and play around with different combinations to see better result. worked very well for HDFC.15 profitable and 3 loss only. 9.8% returns for TATAPOWER.

STRATERGY 7 : Trial testing a stratergy with PSAR alone.

0- Identifying Trending stocks from opportunity universe.
1- Shortlisting stocks based on candle patterns and indicators as above.
2- Using PSAR to determine ENTRY around PSAR reversal.
3- Using PSAR to determine EXIT around PSAR reversal.

In [ ]:
PSAR_trading_df = pd.DataFrame(columns = ['Position','Entry_Date','Entry_Price','Exit_Date','Exit_Price','Holding_Days','P/L'])

def long_trade(i):
    posi = PSAR_trading_df.shape[0]
    PSAR_trading_df.at[posi,'Position'] = 'Long'
    PSAR_trading_df.at[posi,'Entry_Date'] = stock_data_indicator_df.index[i+1]
    PSAR_trading_df.at[posi,'Entry_Price'] = stock_data_indicator_df.loc[stock_data_indicator_df.index[i+1],'Open']
    #global start = i
    
def short_trade(i):
    posi = PSAR_trading_df.shape[0]
    PSAR_trading_df.at[posi,'Position'] = 'Short'
    PSAR_trading_df.at[posi,'Entry_Date'] = stock_data_indicator_df.index[i+1]
    PSAR_trading_df.at[posi,'Entry_Price'] = stock_data_indicator_df.loc[stock_data_indicator_df.index[i+1],'Open']
    #global start = i
    
def exit(i,position):
    if(position == 1):
        long_exit(i)
    elif(position == -1):
        short_exit(i)
    
def long_exit(i):
    posi = PSAR_trading_df.shape[0]-1
    PSAR_trading_df.at[posi,'Exit_Date'] = stock_data_indicator_df.index[i+1]
    PSAR_trading_df.at[posi,'Exit_Price'] = stock_data_indicator_df.loc[stock_data_indicator_df.index[i+1],'Open']
    #PSAR_trading_df.at[posi,'Holding_Days'] = i-start
    PSAR_trading_df.at[posi,'P/L'] = PSAR_trading_df.loc[posi,'Exit_Price'] - PSAR_trading_df.loc[posi,'Entry_Price']
    
def short_exit(i):
    posi = PSAR_trading_df.shape[0]-1
    PSAR_trading_df.at[posi,'Exit_Date'] = stock_data_indicator_df.index[i+1]
    PSAR_trading_df.at[posi,'Exit_Price'] = stock_data_indicator_df.loc[stock_data_indicator_df.index[i+1],'Open']
    #PSAR_trading_df.at[posi,'Holding_Days'] = i-start
    PSAR_trading_df.at[posi,'P/L'] = PSAR_trading_df.loc[posi,'Entry_Price'] - PSAR_trading_df.loc[posi,'Exit_Price']
    

#placing an initial trade before executing the loop based on initial trade
index = 0 
Prior_Trend = stock_data_indicator_df.loc[stock_data_indicator_df.index[index],'Trend']
if(Prior_Trend == -1):
    short_trade(0)
elif(Prior_Trend == 1):
    long_trade(0)

#loop for rest of the trades
for i in range(1,stock_data_indicator_df.shape[0]-1):
    Current_Trend = stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'Trend']
    
    if(Current_Trend == Prior_Trend):
        Prior_Trend = Current_Trend
    elif(Current_Trend != Prior_Trend):
        exit(i,Prior_Trend)
        if(Current_Trend == 1):
            long_trade(i)
            Prior_Trend = 1
        elif(Current_Trend == -1):
            short_trade(i)
            Prior_Trend = -1
        
display(PSAR_trading_df)
print("Total Trades\t"+str(PSAR_trading_df.shape[0]))
print("Profitable Trades\t"+str(PSAR_trading_df[(PSAR_trading_df['P/L']>0)].shape[0]))
print("Non-Profitable Trades\t"+str(PSAR_trading_df[(PSAR_trading_df['P/L']<0)].shape[0]))
print("Net P/L\t"+str(PSAR_trading_df['P/L'].sum()))      

Failed

STRATERGY 8 : Backtesting stratergy using Ichimoku alone

In [ ]:
ichimoku_df = ichimoku_df.drop(['Chikou'],axis=1)
ichimoku_df.dropna(inplace=True)

ichimoku_trading_df = pd.DataFrame(columns=['Position','Entry_Date','Entry_Price','Exit_Date','Exit_Price','Holding_Period','P/L'])

def senkou_check(i):
    if((ichimoku_df.loc[ichimoku_df.index[i],'Senkou_A'] > ichimoku_df.loc[ichimoku_df.index[i],'Senkou_B'])):
        return 1
    elif((ichimoku_df.loc[ichimoku_df.index[i],'Senkou_A'] < ichimoku_df.loc[ichimoku_df.index[i],'Senkou_B'])):
        return -1
    
    
def price_check(i):
    if(ichimoku_df.loc[ichimoku_df.index[i],'Low'] > ichimoku_df.loc[ichimoku_df.index[i],'Senkou_A']):
        return 1
    elif(ichimoku_df.loc[ichimoku_df.index[i],'High'] < ichimoku_df.loc[ichimoku_df.index[i],'Senkou_A']):
        return -1

def Tenkan_and_Kijun_check(i):
    if(ichimoku_df.loc[ichimoku_df.index[i],'Kijun_San'] > ichimoku_df.loc[ichimoku_df.index[i],'Senkou_A']):
        if(ichimoku_df.loc[ichimoku_df.index[i],'Tenkan_San'] > ichimoku_df.loc[ichimoku_df.index[i],'Kijun_San']):
            return 1  
    elif(ichimoku_df.loc[ichimoku_df.index[i],'Kijun_San'] < ichimoku_df.loc[ichimoku_df.index[i],'Senkou_A']):
        if(ichimoku_df.loc[ichimoku_df.index[i],'Tenkan_San'] < ichimoku_df.loc[ichimoku_df.index[i],'Kijun_San']):
            return -1

global t_day

def long_trade(i):
    t_day = i
    posi = ichimoku_trading_df.shape[0]-1
    while((ichimoku_df.loc[ichimoku_df.index[i],'Tenkan_San']>ichimoku_df.loc[ichimoku_df.index[i],'Kijun_San'])&
         (i<ichimoku_df.shape[0]-1)):
        ichimoku_trading_df.at[posi,'Exit_Price'] = ichimoku_df.loc[ichimoku_df.index[i+1],'Open']
        ichimoku_trading_df.at[posi,'Exit_Date'] = stock_data.index[ichimoku_df.index[i+1]] 
        ichimoku_trading_df.at[posi,'P/L'] = ichimoku_trading_df.loc[posi,'Exit_Price'] - ichimoku_trading_df.loc[posi,'Entry_Price']
        i += 1
        start = i
    ichimoku_trading_df.at[posi,'Holding_Period'] = start - t_day
    search(start)
    
def short_trade(i):
    t_day = i
    posi = ichimoku_trading_df.shape[0]-1
    while((ichimoku_df.loc[ichimoku_df.index[i],'Tenkan_San']<ichimoku_df.loc[ichimoku_df.index[i],'Kijun_San'])&
         (i<ichimoku_df.shape[0]-1)):
        ichimoku_trading_df.at[posi,'Exit_Price'] = ichimoku_df.loc[ichimoku_df.index[i+1],'Open']
        ichimoku_trading_df.at[posi,'Exit_Date'] = stock_data.index[ichimoku_df.index[i]] 
        ichimoku_trading_df.at[posi,'P/L'] = ichimoku_trading_df.loc[posi,'Entry_Price'] - ichimoku_trading_df.loc[posi,'Exit_Price']
        i += 1
        start = i
    ichimoku_trading_df.at[posi,'Holding_Period'] = start - t_day
    search(start)

def search(start):
    for i in range(start,ichimoku_df.shape[0]-1):
        if((senkou_check(i) == 1)&
          (price_check(i) == 1)&
          (Tenkan_and_Kijun_check(i) == 1)):
            index = ichimoku_trading_df.shape[0]
            ichimoku_trading_df.at[index,'Entry_Price'] = ichimoku_df.loc[ichimoku_df.index[i],'Close']
            ichimoku_trading_df.at[index,'Entry_Date'] = stock_data.index[ichimoku_df.index[i]]
            ichimoku_trading_df.at[index,'Position'] = 'Long'
            long_trade(i)
            break
#         elif((senkou_check(i) == -1)&
#           (price_check(i) == -1)&
#           (Tenkan_and_Kijun_check(i) == -1)):
#             index = ichimoku_trading_df.shape[0]
#             ichimoku_trading_df.at[index,'Entry_Price'] = ichimoku_df.loc[ichimoku_df.index[i],'Close']
#             ichimoku_trading_df.at[index,'Entry_Date'] = stock_data.index[ichimoku_df.index[i]]
#             ichimoku_trading_df.at[index,'Position'] = 'Short'
#             short_trade(i)
#             break
            
global start
start = 0       
search(start)

print("Total Trades\t"+str(ichimoku_trading_df.shape[0]))
print("Profitable Trades\t"+str(ichimoku_trading_df[(ichimoku_trading_df['P/L']>0)].shape[0]))
print("Non-Profitable Trades\t"+str(ichimoku_trading_df[(ichimoku_trading_df['P/L']<0)].shape[0]))
print("Total Investment\t" + str(ichimoku_trading_df['Entry_Price'].sum()))
print("Current Value\t" + str(ichimoku_trading_df['Exit_Price'].sum()))
print("Net P/L\t"+str(ichimoku_trading_df['P/L'].sum()))
print("Return(%)\t" + str((ichimoku_trading_df['P/L'].sum()/ichimoku_trading_df['Entry_Price'].sum())*100))
ichimoku_trading_df

SUCCESS for PVR. not much for HUL. 1.25% return for HDFC. 3.66% for TATAPOWER

BACKTESTING BEST EXIT STRATERGY with Ichimoku indicating ENTRY.

stock in test: HDFC
stratergy 8: returns 1.25 with ichimoku exit(when tenkan<Kijun)
stratergy 8.1:

STRATERGY 8.1 : Using PSAR for exit.

STRATERGY 9 : Backtesting stratergy using MA trend

In [ ]:
EMA_trading_df = pd.DataFrame(columns=['Position','Entry_Date','Entry_Price','Exit_Date','Exit_Price','Holding_Period','P/L'])

global t_day

def long_trade(i):
    t_day = i
    posi = EMA_trading_df.shape[0]-1
    while((stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'EMA7']>stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'EMA21'])&
         (i < stock_data_indicator_df.shape[0]-1)):
        i += 1
        EMA_trading_df.at[posi,'Exit_Date'] = stock_data_indicator_df.index[i]
        EMA_trading_df.at[posi,'Exit_Price'] = stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'Open']
        EMA_trading_df.at[posi,'P/L'] = EMA_trading_df.loc[posi,'Exit_Price'] - EMA_trading_df.loc[posi,'Entry_Price']
        start = i
    EMA_trading_df.at[posi,'Holding_Period'] = start - t_day
    search(start)
    
def short_trade(i):
    t_day = i
    posi = EMA_trading_df.shape[0]-1
    while((stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'EMA7']<stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'EMA21'])&
         (i < stock_data_indicator_df.shape[0]-1)):
        i += 1
        EMA_trading_df.at[posi,'Exit_Date'] = stock_data_indicator_df.index[i]
        EMA_trading_df.at[posi,'Exit_Price'] = stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'Open']
        EMA_trading_df.at[posi,'P/L'] = EMA_trading_df.loc[posi,'Entry_Price'] - EMA_trading_df.loc[posi,'Exit_Price']
        start = i
    EMA_trading_df.at[posi,'Holding_Period'] = start - t_day
    search(start)

def search(start):
    for i in range(start, stock_data_indicator_df.shape[0]-1):
        if(stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'EMA7'] > stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'EMA21']):
            index = EMA_trading_df.shape[0]
            EMA_trading_df.at[index,'Position'] = 'Long'
            EMA_trading_df.at[index,'Entry_Date'] = stock_data_indicator_df.index[i]
            EMA_trading_df.at[index,'Entry_Price'] = stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'Close']
            long_trade(i)
            break
#         elif(stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'EMA7'] < stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'EMA21']):
#             index = EMA_trading_df.shape[0]
#             EMA_trading_df.at[index,'Position'] = 'Short'
#             EMA_trading_df.at[index,'Entry_Date'] = stock_data_indicator_df.index[i]
#             EMA_trading_df.at[index,'Entry_Price'] = stock_data_indicator_df.loc[stock_data_indicator_df.index[i],'Close']
#             short_trade(i)
#             break
            
global start
start = 0
search(start)
            
print("Total Trades\t"+str(EMA_trading_df.shape[0]))
print("Profitable Trades\t"+str(EMA_trading_df[(EMA_trading_df['P/L']>0)].shape[0]))
print("Non-Profitable Trades\t"+str(EMA_trading_df[(EMA_trading_df['P/L']<0)].shape[0]))
print("Total Investment\t" + str(EMA_trading_df['Entry_Price'].sum()))
print("Current Value\t" + str(EMA_trading_df['Exit_Price'].sum()))
print("Net P/L\t"+str(EMA_trading_df['P/L'].sum()))
print("Return(%)\t" + str((EMA_trading_df['P/L'].sum()/EMA_trading_df['Entry_Price'].sum())*100))
EMA_trading_df


Success for HDFC with 5.3% return for long. 5.5% for tatapower

In [ ]:
STOP

Automated Trading bot

Importing packages for Alice Blue python API

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
from alice_blue import *

Logging in to a profile and displaying it

In [ ]:
#https://develop-api.aliceblueonline.com/dashboard

In [ ]:
user = input("Enter your AliceBlue userid\n")
secret = input("Enter your AliceBlue password\n")
dob = input("Enter your twofa(YOB)\n")
api = input("Enter api\n")
app = input("Enter app id\n")

access_token = AliceBlue.login_and_get_access_token(username=user, password=secret, twoFA=dob, api_secret=api, app_id=app)
alice = AliceBlue(username=user, password=secret, access_token=access_token)
alice = AliceBlue(username=user, password=secret, access_token=access_token, master_contracts_to_download=['NSE','BSE'])
alice_stock_data = alice.get_instrument_by_symbol('NSE', stock_name)

print(alice.get_profile())

Getting live market data

In [ ]:
def event_handler_quote_update(ticks):
    print(ticks['ltp'])
    
def open_callback():
    global socket_opened
    socket_opened = True
    
alice.start_websocket(event_handler_quote_update,open_callback,run_in_background=True)

while True:
    alice.subscribe(alice.get_instrument_by_symbol('NSE', stock_name), LiveFeedType.MARKET_DATA)
    time.sleep(10)

In [ ]:
alice.unsubscribe(alice.get_instrument_by_symbol('NSE', stock_name), LiveFeedType.MARKET_DATA)

In [ ]:
stop

In [ ]:
if(signal == 1):
    print(
        alice.place_order(transaction_type = TransactionType.Buy,
                     instrument = alice.get_instrument_by_symbol('NSE', stock_name),
                     quantity = quantity,
                     order_type = OrderType.Limit,
                     product_type = product_type,
                     price = bid,
                     trigger_price = trigger,
                     stop_loss = stoploss,
                     square_off = target,
                     trailing_sl = trail_stoploss,
                     is_amo = False)
    )

Papertrading in realtime

In [ ]:
if(signal == 1):
    trade()